# 2179 Hard 2179 Count Good Triplets in an Array

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class BIT(object):  # 0-indexed.
    def __init__(self, n):
        self.__bit = [0]*(n+1)  # Extra one for dummy node.

    def add(self, i, val):
        i += 1  # Extra one for dummy node.
        while i < len(self.__bit):
            self.__bit[i] += val
            i += (i & -i)

    def query(self, i):
        i += 1  # Extra one for dummy node.
        ret = 0
        while i > 0:
            ret += self.__bit[i]
            i -= (i & -i)
        return ret


# bit, fenwick tree, combinatorics
class Solution(object):
    def goodTriplets(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: int
        """
        lookup = [0]*len(nums1)
        for i, x in enumerate(nums1):
            lookup[x] = i
        result = 0
        bit = BIT(len(nums1))
        for i, x in enumerate(nums2):
            smaller = bit.query(lookup[x]-1)
            larger = (len(nums1)-(lookup[x]+1))-(i-smaller)
            result += smaller*larger
            bit.add(lookup[x], 1)
        return result

# 2183 Hard 2183 Count Array Pairs Divisible by K

In [ ]:
# Time:  O(nlogk + sqrt(k)^2) = O(nlogk + k)
# Space: O(sqrt(k)), number of factors of k is at most sqrt(k)

import collections


# math, number theory
class Solution(object):
    def countPairs(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def gcd(x, y):
            while y:
                x, y = y, x%y
            return x
    
        cnt = collections.Counter()
        for x in nums:
            cnt[gcd(x, k)] += 1
        result = 0
        for x in cnt.iterkeys():
            for y in cnt.iterkeys():
                if x > y or x*y%k:
                    continue
                result += cnt[x]*cnt[y] if x != y else cnt[x]*(cnt[x]-1)//2
        return result


# Time:  O(nlogk + n * sqrt(k))
# Space: O(sqrt(k)), number of factors of k is at most sqrt(k)
import collections


# math, number theory
class Solution2(object):
    def countPairs(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def gcd(x, y):
            while y:
                x, y = y, x%y
            return x

        result = 0
        gcds = collections.Counter()
        for x in nums:
            gcd_i = gcd(x, k)
            result += sum(cnt for gcd_j, cnt in gcds.iteritems() if gcd_i*gcd_j%k == 0)
            gcds[gcd_i] += 1
        return result

# 2188 Hard 2188 Minimum Time to Finish the Race

In [ ]:
# Time:  O((n + l) * logc)
# Space: O(n + l + logc)

# greedy, dp
class Solution(object):
    def minimumFinishTime(self, tires, changeTime, numLaps):
        """
        :type tires: List[List[int]]
        :type changeTime: int
        :type numLaps: int
        :rtype: int
        """
        def ceil_log2(x):
            return (x-1).bit_length()

        dp = [float("inf")]*ceil_log2(changeTime+1)  # dp[i]: min time to complete i+1 laps without changing a tire
        for f, r in tires:
            total = curr = f
            cnt = 0
            while curr < changeTime+f:  # at worst (f, r) = (1, 2) => 2^(cnt-1) < changeTime+1 => cnt < ceil(log2(changeTime+1))
                dp[cnt] = min(dp[cnt], total)
                curr *= r
                total += curr
                cnt += 1
        dp2 = [float("inf")]*numLaps  # dp2[i]: min time to complete i+1 laps with changing zero or more tires
        for i in xrange(numLaps):
            dp2[i] = min((dp2[i-j-1]+changeTime if i-j-1 >= 0 else 0)+dp[j] for j in xrange(min(i+1, len(dp))))
        return dp2[-1]

# 2193 Hard 2193 Minimum Number of Moves to Make Palindrome

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class BIT(object):  # 0-indexed
    def __init__(self, n):
        self.__bit = [0]*(n+1)

    def add(self, i, val):
        i += 1
        while i < len(self.__bit):
            self.__bit[i] += val
            i += (i & -i)

    def query(self, i):
        i += 1
        ret = 0
        while i > 0:
            ret += self.__bit[i]
            i -= (i & -i)
        return ret


# greedy, bit, fenwick tree
class Solution(object):
    def minMovesToMakePalindrome(self, s):
        """
        :type s: str
        :rtype: int
        """
        idxs = [[] for _ in xrange(26)]
        for i, c in enumerate(s):
            idxs[ord(c)-ord('a')].append(i)
        targets, pairs = [0]*len(s), []
        for c, idx in enumerate(idxs):
            for i in xrange(len(idx)//2):
                pairs.append((idx[i], idx[~i]))
            if len(idx)%2:
                targets[idx[len(idx)//2]] = len(s)//2
        pairs.sort()
        for i, (l, r) in enumerate(pairs):
            targets[l], targets[r] = i, (len(s)-1)-i
        bit = BIT(len(s))
        result = 0
        for i in targets:
            result += i-bit.query(i-1)  # move from bit.query(i-1) to i
            bit.add(i, 1)
        return result


# Time:  O(n^2)
# Space: O(n)
# greedy
class Solution2(object):
    def minMovesToMakePalindrome(self, s):
        """
        :type s: str
        :rtype: int
        """
        s = list(s)
        result = 0
        while s:
            i = s.index(s[-1])
            if i == len(s)-1:
                result += i//2
            else:
                result += i
                s.pop(i)
            s.pop()
        return result

# 2203 Hard 2203 Minimum Weighted Subgraph With the Required Paths

In [ ]:
# Time:  O((|E| + |V|) * log|V|) = O(|E| * log|V|),
#        if we can further to use Fibonacci heap, it would be O(|E| + |V| * log|V|)
# Space: O(|E| + |V|) = O(|E|)

import heapq


# dijkstra's algorithm
class Solution(object):
    def minimumWeight(self, n, edges, src1, src2, dest):
        """
        :type n: int
        :type edges: List[List[int]]
        :type src1: int
        :type src2: int
        :type dest: int
        :rtype: int
        """
        def dijkstra(adj, start):
            best = [float("inf")]*len(adj)
            best[start] = 0
            min_heap = [(0, start)]
            while min_heap:
                curr, u = heapq.heappop(min_heap)
                if best[u] < curr:
                    continue
                for v, w in adj[u]:                
                    if best[v] <= curr+w:
                        continue
                    best[v] = curr+w
                    heapq.heappush(min_heap, (curr+w, v))
            return best
    
        adj1, adj2 = [[[] for _ in xrange(n)] for _ in xrange(2)]
        for u, v, w in edges:
            adj1[u].append((v, w))
            adj2[v].append((u, w))
        dist1 = dijkstra(adj1, src1)
        dist2 = dijkstra(adj1, src2)
        dist3 = dijkstra(adj2, dest)
        result = min(dist1[i]+dist2[i]+dist3[i] for i in xrange(n))
        return result if result != float("inf") else -1

# 2204 Hard 2204 Distance to a Cycle in Undirected Graph

In [ ]:
# Time:  O(|V| + |E|)
# Space: O(|V| + |E|)

# graph, dfs, bfs
class Solution(object):
    def distanceToCycle(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        def cycle(parent, v, u):
            result = [parent[v], v]
            while u != parent[v]:
                result.append(u)
                u = parent[u]
            return result
    
        def iter_dfs(adj):
            stk = [0]
            parent = [-2]*len(adj)
            parent[0] = -1
            while stk:
                u = stk.pop()
                for v in reversed(adj[u]):
                    if parent[v] != -2:
                        if v == parent[u]:
                            continue
                        return cycle(parent, v, u)
                    parent[v] = u
                    stk.append(v)

        def bfs(adj, q):
            result = [-1]*n
            for x in q:
                result[x] = 0
            d = 1
            while q:
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if result[v] != -1:
                            continue
                        result[v] = d
                        new_q.append(v)
                q = new_q
                d += 1
            return result
    
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        return bfs(adj, iter_dfs(adj))

# 2209 Hard 2209 Minimum White Tiles After Covering With Carpets

In [ ]:
# Time:  O(m * n)
# Space: O(m * n)

# dp
class Solution(object):
    def minimumWhiteTiles(self, floor, numCarpets, carpetLen):
        """
        :type floor: str
        :type numCarpets: int
        :type carpetLen: int
        :rtype: int
        """
        dp = [[0]*(numCarpets+1) for _ in xrange(len(floor)+1)]  # dp[i][j] : min number of white tiles in the first i floors with j carpets
        for i in xrange(1, len(dp)):
            dp[i][0] = dp[i-1][0] + int(floor[i-1])
            for j in xrange(1, numCarpets+1):
                dp[i][j] = min(dp[i-1][j] + int(floor[i-1]), dp[max(i-carpetLen, 0)][j-1])
        return dp[-1][-1]

# 2213 Hard 2213 Longest Substring of One Repeating Character

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import itertools


# Template:
# https://github.com/kamyu104/FacebookHackerCup-2021/blob/main/Round%203/auth_ore_ization.py
class SegmentTree(object):
    def __init__(self, N,
                 build_fn=lambda _: float("inf"),
                 query_fn=lambda x, y: x if y is None else min(x, y),
                 update_fn=lambda x: x):
        self.tree = [None]*(2*2**((N-1).bit_length()))
        self.base = len(self.tree)//2
        self.query_fn = query_fn
        self.update_fn = update_fn
        for i in xrange(self.base, self.base+N):
            self.tree[i] = build_fn(i-self.base)
        for i in reversed(xrange(1, self.base)):
            self.tree[i] = query_fn(self.tree[2*i], self.tree[2*i+1])

    def update(self, i, h):
        x = self.base+i
        self.tree[x] = self.update_fn(h)
        while x > 1:
            x //= 2
            self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])


# segment tree
class Solution(object):
    def longestRepeating(self, s, queryCharacters, queryIndices):
        """
        :type s: str
        :type queryCharacters: str
        :type queryIndices: List[int]
        :rtype: List[int]
        """
        LEFT, RIGHT, LEFT_LEN, RIGHT_LEN, LEN, MAX_LEN, SIZE = xrange(7)
        def build(i):
            return update(s[i])

        def update(y):
            result = [0]*SIZE
            result[LEFT] = result[RIGHT] = y
            result[LEN] = result[LEFT_LEN] = result[RIGHT_LEN] = result[MAX_LEN] = 1
            return result

        def query(x, y):
            return x if y is None else \
                   [x[LEFT],
                    y[RIGHT],
                    x[LEFT_LEN]+(y[LEFT_LEN] if x[LEFT_LEN] == x[LEN] and x[RIGHT] == y[LEFT] else 0),
                    y[RIGHT_LEN]+(x[RIGHT_LEN] if y[RIGHT_LEN] == y[LEN] and y[LEFT] == x[RIGHT] else 0),
                    x[LEN]+y[LEN],
                    max(x[MAX_LEN], y[MAX_LEN], x[RIGHT_LEN]+y[LEFT_LEN] if x[RIGHT] == y[LEFT] else 0)]
        
        result = []
        st = SegmentTree(len(s), build_fn=build, query_fn=query, update_fn=update)
        for c, i in itertools.izip(queryCharacters, queryIndices):
            st.update(i, c)
            result.append(st.tree[1][MAX_LEN])
        return result


# Time:  O(nlogn)
# Space: O(n)
import itertools


# Template:
# https://github.com/kamyu104/FacebookHackerCup-2021/blob/main/Round%203/auth_ore_ization.py
class SegmentTree2(object):
    def __init__(self, N,
                 build_fn=lambda _: float("inf"),
                 query_fn=lambda x, y: y if x is None else x if y is None else min(x, y),
                 update_fn=lambda x: x):
        self.tree = [None]*(2*2**((N-1).bit_length()))
        self.base = len(self.tree)//2
        self.query_fn = query_fn
        self.update_fn = update_fn
        for i in xrange(self.base, self.base+N):
            self.tree[i] = build_fn(i-self.base)
        for i in reversed(xrange(1, self.base)):
            self.tree[i] = query_fn(self.tree[2*i], self.tree[2*i+1])

    def update(self, i, h):
        x = self.base+i
        self.tree[x] = self.update_fn(h)
        while x > 1:
            x //= 2
            self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])

    def query(self, L, R):
        if L > R:
            return None
        L += self.base
        R += self.base
        left = right = None
        while L <= R:
            if L & 1:
                left = self.query_fn(left, self.tree[L])
                L += 1
            if R & 1 == 0:
                right = self.query_fn(self.tree[R], right)
                R -= 1
            L //= 2
            R //= 2
        return self.query_fn(left, right)


# segment tree
class Solution2(object):
    def longestRepeating(self, s, queryCharacters, queryIndices):
        """
        :type s: str
        :type queryCharacters: str
        :type queryIndices: List[int]
        :rtype: List[int]
        """
        LEFT, RIGHT, LEFT_LEN, RIGHT_LEN, LEN, MAX_LEN, SIZE = xrange(7)
        def build(i):
            return update(s[i])

        def update(y):
            result = [0]*SIZE
            result[LEN] = result[LEFT_LEN] = result[RIGHT_LEN] = result[MAX_LEN] = 1
            result[LEFT] = result[RIGHT] = y
            return result

        def query(x, y):
            return y if x is None else x if y is None else \
                   [x[LEFT],
                    y[RIGHT],
                    x[LEFT_LEN]+(y[LEFT_LEN] if x[LEFT_LEN] == x[LEN] and x[RIGHT] == y[LEFT] else 0),
                    y[RIGHT_LEN]+(x[RIGHT_LEN] if y[RIGHT_LEN] == y[LEN] and y[LEFT] == x[RIGHT] else 0),
                    x[LEN]+y[LEN],
                    max(x[MAX_LEN], y[MAX_LEN], x[RIGHT_LEN]+y[LEFT_LEN] if x[RIGHT] == y[LEFT] else 0)]
        
        result = []
        st = SegmentTree2(len(s), build_fn=build, query_fn=query, update_fn=update)
        for c, i in itertools.izip(queryCharacters, queryIndices):
            st.update(i, c)
            result.append(st.query(0, len(s)-1)[MAX_LEN])
        return result

# 2218 Hard 2218 Maximum Value of K Coins From Piles

In [ ]:
# Time:  O(min(n * k^2, m * k)), m = sum(len(pile) for pile in piles)
# Space: O(k)

# dp
class Solution(object):
    def maxValueOfCoins(self, piles, k):
        """
        :type piles: List[List[int]]
        :type k: int
        :rtype: int
        """
        dp = [0]
        for pile in piles:
            new_dp = [0]*min(len(dp)+len(pile), k+1)
            for i in xrange(len(dp)):
                curr = 0
                for j in xrange(min(k-i, len(pile))+1):
                    new_dp[i+j] = max(new_dp[i+j], dp[i]+curr)
                    curr += pile[j] if j < len(pile) else 0
            dp = new_dp
        return dp[-1]

# 2223 Hard 2223 Sum of Scores of Built Strings

In [ ]:
# Time:  O(n)
# Space: O(n)

# z-function
class Solution(object):
    def sumScores(self, s):
        """
        :type s: str
        :rtype: int
        """
        # Template: https://cp-algorithms.com/string/z-function.html
        def z_function(s):  # Time: O(n), Space: O(n)
            z = [0]*len(s)
            l, r = 0, 0
            for i in xrange(1, len(z)):
                if i <= r:
                    z[i] = min(r-i+1, z[i-l])
                while i+z[i] < len(z) and s[z[i]] == s[i+z[i]]:
                    z[i] += 1
                if i+z[i]-1 > r:
                    l, r = i, i+z[i]-1
            return z

        z = z_function(s)
        z[0] = len(s)
        return sum(z)

# 2227 Hard 2227 Encrypt and Decrypt Strings

In [ ]:
# Time:  ctor:    O(m + d), m is len(keys), d is sum(len(x) for x in dictionary)
#        encrypt: O(n)
#        decrypt: O(n)
# Space: O(m + d)

import collections
import itertools


# freq table
class Encrypter(object):

    def __init__(self, keys, values, dictionary):
        """
        :type keys: List[str]
        :type values: List[str]
        :type dictionary: List[str]
        """
        self.__lookup = {k: v for k, v in itertools.izip(keys, values)}
        self.__cnt = collections.Counter(self.encrypt(x) for x in dictionary)
        
    def encrypt(self, word1):
        """
        :type word1: str
        :rtype: str
        """
        if any(c not in self.__lookup for c in word1):
            return ""
        return "".join(self.__lookup[c] for c in word1)

    def decrypt(self, word2):
        """
        :type word2: str
        :rtype: int
        """
        return self.__cnt[word2]

# 2234 Hard 2234 Maximum Total Beauty of the Gardens

In [ ]:
# Time:  O(nlogn)
# Space: O(1)

import bisect


# sort, prefix sum, greedy, two pointers, improved from solution3
class Solution(object):
    def maximumBeauty(self, flowers, newFlowers, target, full, partial):
        """
        :type flowers: List[int]
        :type newFlowers: int
        :type target: int
        :type full: int
        :type partial: int
        :rtype: int
        """
        flowers.sort()
        n = bisect.bisect_left(flowers, target)
        prefix, suffix = 0, sum(flowers[i] for i in xrange(n))
        result = left = 0
        for right in xrange(n+1):
            if right:
                suffix -= flowers[right-1]
            total = newFlowers-((n-right)*target-suffix)
            if total < 0:
                continue
            while not (left == right or (left and (total+prefix)//left <= flowers[left])):
                prefix += flowers[left]
                left += 1
            mn = min((total+prefix)//left if left else 0, target-1)
            result = max(result, mn*partial+(len(flowers)-right)*full)
        return result


# Time:  O(nlogn)
# Space: O(1)
import bisect


# sort, prefix sum, greedy, two pointers, improved from solution4
class Solution2(object):
    def maximumBeauty(self, flowers, newFlowers, target, full, partial):
        """
        :type flowers: List[int]
        :type newFlowers: int
        :type target: int
        :type full: int
        :type partial: int
        :rtype: int
        """
        flowers.sort()
        n = bisect.bisect_left(flowers, target)
        prefix = [0]*(n+1)
        for i in xrange(n):
            prefix[i+1] = prefix[i]+flowers[i]
        result = suffix = 0
        left = n
        for right in reversed(xrange(n+1)):
            if right != n:
                suffix += flowers[right]
            total = newFlowers-((n-right)*target-suffix)
            if total < 0:
                continue
            left = min(left, right)
            while not (left == 0 or (prefix[left]-prefix[left-1])*left-prefix[left] <= total):
                left -= 1
            mn = min((total+prefix[left])//left if left else 0, target-1)
            result = max(result, mn*partial+(len(flowers)-right)*full)
        return result


# Time:  O(nlogn)
# Space: O(n)
import bisect


# sort, prefix sum, greedy, binary search
class Solution3(object):
    def maximumBeauty(self, flowers, newFlowers, target, full, partial):
        """
        :type flowers: List[int]
        :type newFlowers: int
        :type target: int
        :type full: int
        :type partial: int
        :rtype: int
        """
        def check(prefix, total, x):
            return x and (total+prefix[x])//x <= prefix[x+1]-prefix[x]

        def binary_search(prefix, total, left, right):
            while left <= right:
                mid = left+(right-left)//2
                if check(prefix, total, mid):
                    right = mid-1
                else:
                    left = mid+1
            return left
    
        flowers.sort()
        n = bisect.bisect_left(flowers, target)
        prefix = [0]*(n+1)
        for i in xrange(n):
            prefix[i+1] = prefix[i]+flowers[i]
        suffix = sum(flowers[i] for i in xrange(n))
        result = left = 0
        for right in xrange(n+1):
            if right:
                suffix -= flowers[right-1]
            total = newFlowers-((n-right)*target-suffix)
            if total < 0:
                continue
            left = binary_search(prefix, total, 0, right-1)
            mn = min((total+prefix[left])//left if left else 0, target-1)
            result = max(result, mn*partial+(len(flowers)-right)*full)
        return result


# Time:  O(nlogn)
# Space: O(n)
import bisect


# sort, prefix sum, greedy, binary search
class Solution4(object):
    def maximumBeauty(self, flowers, newFlowers, target, full, partial):
        """
        :type flowers: List[int]
        :type newFlowers: int
        :type target: int
        :type full: int
        :type partial: int
        :rtype: int
        """
        def check(prefix, total, x):
            return (prefix[x]-prefix[x-1])*x-prefix[x] <= total

        def binary_search_right(prefix, total, left, right):
            while left <= right:
                mid = left+(right-left)//2
                if not check(prefix, total, mid):
                    right = mid-1
                else:
                    left = mid+1
            return right
    
        flowers.sort()
        n = bisect.bisect_left(flowers, target)
        prefix = [0]*(n+1)
        for i in xrange(n):
            prefix[i+1] = prefix[i]+flowers[i]
        result = suffix = 0
        left = n
        for right in reversed(xrange(n+1)):
            if right != n:
                suffix += flowers[right]
            total = newFlowers-((n-right)*target-suffix)
            if total < 0:
                break
            left = binary_search_right(prefix, total, 1, right)
            mn = min((total+prefix[left])//left if left else 0, target-1)
            result = max(result, mn*partial+(len(flowers)-right)*full)
        return result

# 2242 Hard 2242 Maximum Score of a Node Sequence

In [ ]:
# Time:  O(|V| + |E|)
# Space: O(|V|)

import heapq


# graph
class Solution(object):
    def maximumScore(self, scores, edges):
        """
        :type scores: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def find_top3(scores, x, top3):
            heapq.heappush(top3, (scores[x], x))
            if len(top3) > 3:
                heapq.heappop(top3)

        top3 = [[] for _ in xrange(len(scores))]
        for a, b in edges:
            find_top3(scores, b, top3[a])
            find_top3(scores, a, top3[b])
        result = -1
        for a, b in edges:
            for _, c in top3[a]:
                if c == b:
                    continue
                for _, d in top3[b]:
                    if d == a or d == c:
                        continue
                    result = max(result, sum(scores[x] for x in (a, b, c, d)))
        return result

# 2246 Hard 2246 Longest Path With Different Adjacent Characters

In [ ]:
# Time:  O(n)
# Space: O(w)

import collections


# tree, bfs, topological sort
class Solution(object):
    def longestPath(self, parent, s):
        """
        :type parent: List[int]
        :type s: str
        :rtype: int
        """
        def topological_sort(s, adj, in_degree):
            result = 1
            top2 = collections.defaultdict(lambda:[0]*2)
            q =  [(i, 1) for i, d in enumerate(in_degree) if not d]
            while q:
                new_q = []
                for (u, l) in q:
                    for v in adj[u]:
                        if s[v] != s[u]:
                            if l > top2[v][0]:
                                top2[v][0], top2[v][1] = l, top2[v][0]
                            elif l > top2[v][1]:
                                top2[v][1] = l
                        in_degree[v] -= 1
                        if in_degree[v]:
                            continue
                        new_q.append((v, top2[v][0]+1))
                        result = max(result, top2[v][0]+top2[v][1]+1)
                        del top2[v]
                q = new_q
            return result

        adj = [[] for _ in xrange(len(s))]
        in_degree = [0]*len(s)
        for i in xrange(1, len(parent)):
            adj[i].append(parent[i])
            in_degree[parent[i]] += 1
        return topological_sort(s, adj, in_degree)


# Time:  O(n)
# Space: O(h)
# tree, dfs
class Solution2(object):
    def longestPath(self, parent, s):
        """
        :type parent: List[int]
        :type s: str
        :rtype: int
        """
        def iter_dfs(s, adj):
            result = 0
            stk = [(1, (0, [0]))]
            while stk:
                step, args = stk.pop()
                if step == 1:
                    u, ret = args
                    top2 = [0]*2
                    stk.append((4, (top2, ret)))
                    stk.append((2, (u, 0, top2, ret)))
                elif step == 2:
                    u, i, top2, ret = args
                    if i == len(adj[u]):
                        continue
                    ret2 = [0]
                    stk.append((3, (u, i, top2, ret2)))
                    stk.append((1, (adj[u][i], ret2))) 
                elif step == 3:
                    u, i, top2, ret2 = args
                    if s[adj[u][i]] != s[u]:
                        if ret2[0] > top2[0]:
                            top2[0], top2[1] = ret2[0], top2[0]
                        elif ret2[0] > top2[1]:
                            top2[1] = ret2[0]
                    stk.append((2, (u, i+1, top2, ret)))
                elif step == 4:
                    top2, ret = args
                    result = max(result, top2[0]+top2[1]+1)
                    ret[0] = top2[0]+1
            return result
    
        
        adj = [[] for _ in xrange(len(s))]
        for i in xrange(1, len(parent)):
            adj[parent[i]].append(i)
        return iter_dfs(s, adj)
    

# Time:  O(n)
# Space: O(h)
# tree, dfs
class Solution3(object):
    def longestPath(self, parent, s):
        """
        :type parent: List[int]
        :type s: str
        :rtype: int
        """
        def dfs(s, adj, u, result):
            top2 = [0]*2
            for v in adj[u]:
                l = dfs(s, adj, v, result)
                if s[v] == s[u]:
                    continue
                if l > top2[0]:
                    top2[0], top2[1] = l, top2[0]
                elif l > top2[1]:
                    top2[1] = l
            result[0] = max(result[0], top2[0]+top2[1]+1)
            return top2[0]+1
    
        
        adj = [[] for _ in xrange(len(s))]
        for i in xrange(1, len(parent)):
            adj[parent[i]].append(i)
        result = [0]
        dfs(s, adj, 0, result)
        return result[0]
    

# 2247 Hard 2247 Maximum Cost of Trip With K Highways

In [ ]:
# Time:  O(n^2 * 2^n)
# Space: O(n * 2^n)

import itertools


# combination based dp
class Solution(object):
    def maximumCost(self, n, highways, k):
        """
        :type n: int
        :type highways: List[List[int]]
        :type k: int
        :rtype: int
        """
        if k+1 > n:  # optionally optimize
            return -1
        adj = [[] for _ in xrange(n)]
        for c1, c2, t in highways:
            adj[c1].append((c2, t))
            adj[c2].append((c1, t))
        result = -1 if k != 1 else 0
        dp = [[0, []] for _ in xrange((1<<n))]
        for i in xrange(n):
            dp[1<<i][1].append(i)
        for cnt in xrange(1, n+1):
            for choice in itertools.combinations(xrange(n), cnt):
                mask = reduce(lambda x, y:x|(1<<y), choice, 0)
                total, lasts = dp[mask]
                for u in lasts:
                    for v, t in adj[u]:
                        if mask&(1<<v):
                            continue
                        new_mask = mask|(1<<v)
                        if total+t < dp[new_mask][0]:
                            continue
                        if total+t == dp[new_mask][0]:
                            dp[new_mask][1].append(v)
                            continue
                        dp[new_mask][0] = total+t
                        dp[new_mask][1] = [v]
                        if bin(mask).count('1') == k:
                            result = max(result, dp[new_mask][0])
        return result


# Time:  O(n^2 * 2^n)
# Space: O(n * 2^n)
# bfs based dp
class Solution2(object):
    def maximumCost(self, n, highways, k):
        """
        :type n: int
        :type highways: List[List[int]]
        :type k: int
        :rtype: int
        """
        if k+1 > n:  # required to optimize, otherwise, TLE or MLE
            return -1
        adj = [[] for _ in xrange(n)]
        for c1, c2, t in highways:
            adj[c1].append((c2, t))
            adj[c2].append((c1, t))
        result = -1
        dp = [(u, 1<<u, 0) for u in xrange(n)]
        while dp:
            new_dp = []
            for u, mask, total in dp:
                if bin(mask).count('1') == k+1:
                    result = max(result, total)
                for v, t in adj[u]:
                    if mask&(1<<v) == 0:
                        new_dp.append((v, mask|(1<<v), total+t))
            dp = new_dp
        return result

# 2251 Hard 2251 Number of Flowers in Full Bloom

In [ ]:
# Time:  O(nlogn + mlogn)
# Space: O(n)

import bisect


# line sweep, binary search
class Solution(object):
    def fullBloomFlowers(self, flowers, persons):
        """
        :type flowers: List[List[int]]
        :type persons: List[int]
        :rtype: List[int]
        """
        cnt = collections.Counter()
        for s, e in flowers:
            cnt[s] += 1
            cnt[e+1] -= 1
        events = sorted(cnt.iterkeys())
        prefix = [0]
        for x in events:
            prefix.append(prefix[-1]+cnt[x])
        return [prefix[bisect.bisect_right(events, t)] for t in persons]


# Time:  O(nlogn + mlogn)
# Space: O(n)
import bisect


# binary search
class Solution(object):
    def fullBloomFlowers(self, flowers, persons):
        """
        :type flowers: List[List[int]]
        :type persons: List[int]
        :rtype: List[int]
        """
        starts, ends = [], []
        for s, e in flowers:
            starts.append(s)
            ends.append(e+1)
        starts.sort()
        ends.sort()
        return [bisect.bisect_right(starts, t)-bisect.bisect_right(ends, t) for t in persons]

# 2254 Hard 2254 Design Video Sharing Platform

In [ ]:
# Time:  ctor:                O(1)
#        upload:              O(logn+l)
#        remove:              O(logn)
#        like:                O(1)
#        dislike:             O(1)
#        view:                O(l)
#        getLikesAndDislikes: O(1)
#        getViews:            O(1)
# Space: O(n * l), n = len(videos), l = max(len(v) for v in videos) 

import heapq


# design, heap
class VideoSharingPlatform(object):

    def __init__(self):
        self.__avails = []
        self.__videos = []
        self.__likes = []
        self.__dislikes = []
        self.__views = []

    def upload(self, video):
        """
        :type video: str
        :rtype: int
        """
        if self.__avails:
            i = heapq.heappop(self.__avails)
        else:
            i = len(self.__videos)
            self.__videos.append(None)
            self.__likes.append(0)
            self.__dislikes.append(0)
            self.__views.append(0)
        self.__videos[i] = video
        return i
        
    def remove(self, videoId):
        """
        :type videoId: int
        :rtype: None
        """
        if videoId >= len(self.__videos) or not self.__videos[videoId]:
            return
        heapq.heappush(self.__avails, videoId)
        self.__videos[videoId] = None
        self.__likes[videoId] = self.__dislikes[videoId] = self.__views[videoId] = 0
        
    def watch(self, videoId, startMinute, endMinute):
        """
        :type videoId: int
        :type startMinute: int
        :type endMinute: int
        :rtype: str
        """
        if videoId >= len(self.__videos) or not self.__videos[videoId]:
            return "-1"
        self.__views[videoId] += 1
        return self.__videos[videoId][startMinute:endMinute+1]

    def like(self, videoId):
        """
        :type videoId: int
        :rtype: None
        """
        if videoId >= len(self.__videos) or not self.__videos[videoId]:
            return
        self.__likes[videoId] += 1

    def dislike(self, videoId):
        """
        :type videoId: int
        :rtype: None
        """
        if videoId >= len(self.__videos) or not self.__videos[videoId]:
            return
        self.__dislikes[videoId] += 1

    def getLikesAndDislikes(self, videoId):
        """
        :type videoId: int
        :rtype: List[int]
        """
        if videoId >= len(self.__videos) or not self.__videos[videoId]:
            return [-1]
        return [self.__likes[videoId], self.__dislikes[videoId]]

    def getViews(self, videoId):
        """
        :type videoId: int
        :rtype: int
        """
        if videoId >= len(self.__videos) or not self.__videos[videoId]:
            return -1
        return self.__views[videoId]

# 2258 Hard 2258 Escape the Spreading Fire

In [ ]:
# Time:  O(m * n)
# Space: O(m * n)

import collections


# bfs
class Solution(object):
    def maximumMinutes(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        GRASS, FIRE, WALL, PERSON = range(4)
        INF = 10**9
        def bfs(grid):
            time = collections.defaultdict(int)
            d = 0
            q = [(r, c, FIRE) for r in xrange(len(grid)) for c in xrange(len(grid[0])) if grid[r][c] == FIRE]
            q.append((0, 0, PERSON))
            while q:
                new_q = []
                for r, c, t in q:
                    for dr, dc in DIRECTIONS:
                        nr, nc = r+dr, c+dc
                        if not (0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and
                                grid[nr][nc] != WALL and
                                ((t == FIRE and grid[nr][nc] != FIRE) or
                                 (t == PERSON and (grid[nr][nc] == GRASS or (grid[nr][nc] == FIRE and (nr, nc) == (len(grid)-1, len(grid[0])-1) and d+1 == time[FIRE, nr, nc]))))):
                            continue
                        if grid[nr][nc] != FIRE:
                            grid[nr][nc] = t
                        if (nr, nc) in ((len(grid)-1, len(grid[0])-1), (len(grid)-1, len(grid[0])-2), (len(grid)-2, len(grid[0])-1)):
                            time[t, nr, nc] = d+1
                        new_q.append((nr, nc, t))
                q = new_q
                d += 1
            return time

        time = bfs(grid)
        if not time[PERSON, len(grid)-1, len(grid[0])-1]:
            return -1
        if not time[FIRE, len(grid)-1, len(grid[0])-1]:
            return INF
        diff = time[FIRE, len(grid)-1, len(grid[0])-1]-time[PERSON, len(grid)-1, len(grid[0])-1]
        return diff if diff+2 in (time[FIRE, len(grid)-1, len(grid[0])-2]-time[PERSON, len(grid)-1, len(grid[0])-2],
                                  time[FIRE, len(grid)-2, len(grid[0])-1]-time[PERSON, len(grid)-2, len(grid[0])-1]) else diff-1


# Time:  O(m * n)
# Space: O(m * n)
# bfs
class Solution2(object):
    def maximumMinutes(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        FIRE, WALL, PERSON = range(1, 4)
        INF = 10**9
        def bfs(grid):
            time = {FIRE:[[INF]*len(grid[0]) for _ in xrange(len(grid))],
                    PERSON:[[INF]*len(grid[0]) for _ in xrange(len(grid))]}
            d = 0
            q = [(r, c, FIRE) for r in xrange(len(grid)) for c in xrange(len(grid[0])) if grid[r][c] == FIRE]
            q.append((0, 0, PERSON))
            for r, c, t in q:
                time[t][r][c] = d
            while q:
                new_q = []
                for r, c, t in q:
                    for dr, dc in DIRECTIONS:
                        nr, nc = r+dr, c+dc
                        if not (0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and
                                grid[nr][nc] != WALL and time[t][nr][nc] == INF and
                                (t == FIRE or
                                 d+1 < time[FIRE][nr][nc] or (d+1 == time[FIRE][nr][nc] and (nr, nc) == (len(grid)-1, len(grid[0])-1)))):
                            continue
                        time[t][nr][nc] = d+1
                        new_q.append((nr, nc, t))
                q = new_q
                d += 1
            return time

        time = bfs(grid)
        if time[PERSON][-1][-1] == INF:
            return -1
        if time[FIRE][-1][-1] == INF:
            return INF
        diff = time[FIRE][-1][-1]-time[PERSON][-1][-1]
        return diff if diff+2 in (time[FIRE][-1][-2]-time[PERSON][-1][-2], time[FIRE][-2][-1]-time[PERSON][-2][-1]) else diff-1

# 2262 Hard 2262 Total Appeal of A String

In [ ]:
# Time:  O(n)
# Space: O(26)

# combinatorics
class Solution(object):
    def appealSum(self, s):
        """
        :type s: str
        :rtype: int
        """
        result = curr = 0
        lookup = [-1]*26
        for i, c in enumerate(s):
            result += (i-lookup[ord(c)-ord('a')])*(len(s)-i)
            lookup[ord(c)-ord('a')] = i
        return result


# Time:  O(n)
# Space: O(26)
# counting
class Solution2(object):
    def appealSum(self, s):
        """
        :type s: str
        :rtype: int
        """
        result = cnt = 0
        lookup = [-1]*26
        for i, c in enumerate(s):
            cnt += i-lookup[ord(c)-ord('a')]
            lookup[ord(c)-ord('a')] = i
            result += cnt
        return result

# 2267 Hard 2267 Check if There Is a Valid Parentheses String Path

In [ ]:
# Time:  O((m * n) * (m + n) / 32)
# Space: O(n * (m + n) / 32)

# dp with bitsets
class Solution(object):
    def hasValidPath(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: bool
        """
        if (len(grid)+len(grid[0])-1)%2:
            return False
        dp = [0]*(len(grid[0])+1)
        for i in xrange(len(grid)):
            dp[0] = int(not i)
            for j in xrange(len(grid[0])):
                dp[j+1] = (dp[j]|dp[j+1])<<1 if grid[i][j] == '(' else (dp[j]|dp[j+1])>>1
        return dp[-1]&1


# Time:  O(m * n)
# Space: O(n)
# dp, optimized from solution1 (wrong answer)
class Solution_WA(object):
    def hasValidPath(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: bool
        """
        if (len(grid)+len(grid[0])-1)%2:
            return False
        dp = [[float("inf"), float("-inf")] for _ in xrange(len(grid[0])+1)]
        for i in xrange(len(grid)):
            dp[0] = [0, 0] if not i else [float("inf"), float("-inf")]
            for j in xrange(len(grid[0])):
                d = 1 if grid[i][j] == '(' else -1
                dp[j+1] = [min(dp[j+1][0], dp[j][0])+d, max(dp[j+1][1], dp[j][1])+d]
                # bitset pattern is like xxx1010101xxxx (in fact, it is not always true in this problem where some paths are invalid)
                if dp[j+1][1] < 0:
                    dp[j+1] = [float("inf"), float("-inf")]
                else:
                    dp[j+1][0] = max(dp[j+1][0], dp[j+1][1]%2)
        return dp[-1][0] == 0

# 2272 Hard 2272 Substring With Largest Variance

In [ ]:
# Time:  O(a^2 * n), a is the size of alphabets
# Space: O(a)

import itertools


# kadane's algorithm
class Solution(object):
    def largestVariance(self, s):
        """
        :type s: str
        :rtype: int
        """
        def modified_kadane(a, x, y):
            result = curr = 0
            lookup = [0]*2
            remain = [a.count(x), a.count(y)]
            for c in a:
                if c not in (x, y):
                    continue
                lookup[c != x] = 1
                remain[c != x] -= 1
                curr += 1 if c == x else -1
                if curr < 0 and remain[0] and remain[1]:
                    curr = lookup[0] = lookup[1] = 0  # reset states if the remain has both x, y
                if lookup[0] and lookup[1]:
                    result = max(result, curr)  # update result if x, y both exist
            return result

        alphabets = set(s)
        return max(modified_kadane(s, x, y) for x, y in itertools.permutations(alphabets, 2)) if len(alphabets) >= 2 else 0

# 2276 Hard 2276 Count Integers in Intervals

In [ ]:
# Time:  ctor:  O(1)
#        add:   O(logn), amortized
#        count: O(1)
# Space: O(n)

from sortedcontainers import SortedList


# design, sortedlist
class CountIntervals(object):

    def __init__(self):
        self.__sl = SortedList()
        self.__cnt = 0

    def add(self, left, right):
        """
        :type left: int
        :type right: int
        :rtype: None
        """
        i = self.__sl.bisect_right((left,))
        if i-1 >= 0 and self.__sl[i-1][1]+1 >= left:
            i -= 1
            left = self.__sl[i][0]
        to_remove = []
        for i in xrange(i, len(self.__sl)):
            if not (right+1 >= self.__sl[i][0]):
                break
            right = max(right, self.__sl[i][1])
            self.__cnt -= self.__sl[i][1]-self.__sl[i][0]+1
            to_remove.append(i)
        while to_remove:
            del self.__sl[to_remove.pop()]
        self.__sl.add((left, right))
        self.__cnt += right-left+1

    def count(self):
        """
        :rtype: int
        """
        return self.__cnt

# 2277 Hard 2277 Closest Node to Path in Tree

In [ ]:
# Time:  O(n + q)
# Space: O(n)

import collections
from functools import partial


# Template:
# https://github.com/kamyu104/GoogleKickStart-2021/blob/main/Round%20H/dependent_events3.py
# Tarjan's Offline LCA Algorithm
class UnionFind(object):  # Time: O(n * alpha(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n
        self.ancestor = range(n)  # added

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x, y = self.find_set(x), self.find_set(y)
        if x == y:
            return False
        if self.rank[x] > self.rank[y]:  # union by rank
            x, y = y, x
        self.set[x] = self.set[y]
        if self.rank[x] == self.rank[y]:
            self.rank[y] += 1
        return True

    def find_ancestor_of_set(self, x):  # added
        return self.ancestor[self.find_set(x)]

    def update_ancestor_of_set(self, x):  # added
        self.ancestor[self.find_set(x)] = x


class TreeInfos(object):  # Time: O(N), Space: O(N + Q), N is the number of nodes
    def __init__(self, children, pairs):
        def preprocess(curr, parent):
            # depth of the node i
            D[curr] = 1 if parent == -1 else D[parent]+1

        def divide(curr, parent):
            stk.append(partial(postprocess, curr))
            for i in reversed(xrange(len(children[curr]))):
                child = children[curr][i]
                if child == parent:
                    continue
                stk.append(partial(conquer, child, curr))
                stk.append(partial(divide, child, curr))
            stk.append(partial(preprocess, curr, parent))

        def conquer(curr, parent):
            uf.union_set(curr, parent)
            uf.update_ancestor_of_set(parent)

        def postprocess(u):
            lookup[u] = True
            for v in pairs[u]:
                if not lookup[v]:
                    continue
                lca[min(u, v), max(u, v)] = uf.find_ancestor_of_set(v)

        N = len(children)
        D, uf, lca = [0]*N, UnionFind(N), {}
        stk, lookup = [], [False]*N
        stk.append(partial(divide, 0, -1))
        while stk:
            stk.pop()()
        self.D, self.lca = D, lca


# Tarjan's Offline LCA Algorithm
class Solution(object):
    def closestNode(self, n, edges, query):
        """
        :type n: int
        :type edges: List[List[int]]
        :type query: List[List[int]]
        :rtype: List[int]
        """
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v), adj[v].append(u)
        pairs = collections.defaultdict(set)
        for start, end, node in query:
            pairs[start].add(end), pairs[end].add(start)
            pairs[start].add(node), pairs[node].add(start)
            pairs[end].add(node), pairs[node].add(end)
        tree_infos = TreeInfos(adj, pairs)
        return [max((tree_infos.lca[min(x, y), max(x, y)] for x, y in ((start, end), (start, node), (end, node))), key=lambda x: tree_infos.D[x]) for start, end, node in query]


# Time:  O(nlogn + qlogn)
# Space: O(nlogn)
from functools import partial


# Template:
# https://github.com/kamyu104/GoogleKickStart-2021/blob/main/Round%20H/dependent_events2.py
class TreeInfos2(object):  # Time: O(NlogN), Space: O(NlogN), N is the number of nodes
    def __init__(self, children):  # modified
        def preprocess(curr, parent):
            # depth of the node i
            D[curr] = 1 if parent == -1 else D[parent]+1
            # ancestors of the node i
            if parent != -1:
                P[curr].append(parent)
            i = 0
            while i < len(P[curr]) and i < len(P[P[curr][i]]):
                P[curr].append(P[P[curr][i]][i])
                i += 1
            # the subtree of the node i is represented by traversal index L[i]..R[i]
            C[0] += 1
            L[curr] = C[0]

        def divide(curr, parent):
            stk.append(partial(postprocess, curr))
            for i in reversed(xrange(len(children[curr]))):
                child = children[curr][i]
                if child == parent:
                    continue
                stk.append(partial(divide, child, curr))
            stk.append(partial(preprocess, curr, parent))

        def postprocess(curr):
            R[curr] = C[0]

        N = len(children)
        L, R, D, P, C = [0]*N, [0]*N, [0]*N, [[] for _ in xrange(N)], [-1]
        stk = []
        stk.append(partial(divide, 0, -1))
        while stk:
            stk.pop()()
        assert(C[0] == N-1)
        self.L, self.R, self.D, self.P = L, R, D, P

    # Template:
    # https://github.com/kamyu104/FacebookHackerCup-2019/blob/master/Final%20Round/little_boat_on_the_sea.py
    def is_ancestor(self, a, b):  # includes itself
        return self.L[a] <= self.L[b] <= self.R[b] <= self.R[a]

    def lca(self, a, b):
        if self.D[a] > self.D[b]:
            a, b = b, a
        if self.is_ancestor(a, b):
            return a
        for i in reversed(xrange(len(self.P[a]))):  # O(logN)
            if i < len(self.P[a]) and not self.is_ancestor(self.P[a][i], b):
                a = self.P[a][i]
        return self.P[a][0]


# binary lifting (online lca algorithm)
class Solution2(object):
    def closestNode(self, n, edges, query):
        """
        :type n: int
        :type edges: List[List[int]]
        :type query: List[List[int]]
        :rtype: List[int]
        """
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v), adj[v].append(u)
        tree_infos = TreeInfos2(adj)
        return [max((tree_infos.lca(x, y) for x, y in ((start, end), (start, node), (end, node))), key=lambda x: tree_infos.D[x]) for start, end, node in query]


# Time:  O(n + q * h)
# Space: O(n)
from functools import partial


# Template:
# https://github.com/kamyu104/GoogleKickStart-2021/blob/main/Round%20H/dependent_events2.py
class TreeInfos3(object):  # Time: O(N), Space: O(N), N is the number of nodes
    def __init__(self, children):  # modified
        def preprocess(curr, parent):
            # depth of the node i
            D[curr] = 1 if parent == -1 else D[parent]+1
            # ancestors of the node i
            P[curr] = parent

        def divide(curr, parent):
            for i in reversed(xrange(len(children[curr]))):
                child = children[curr][i]
                if child == parent:
                    continue
                stk.append(partial(divide, child, curr))
            stk.append(partial(preprocess, curr, parent))

        N = len(children)
        D, P = [0]*N, [0]*N
        stk = []
        stk.append(partial(divide, 0, -1))
        while stk:
            stk.pop()()
        self.D, self.P = D, P

    def lca(self, a, b):  # Time: O(logh)
        while self.D[a] > self.D[b]:
            a = self.P[a]
        while self.D[a] < self.D[b]:
            b = self.P[b]
        while a != b:
            a, b = self.P[a], self.P[b]
        return a


# lca
class Solution3(object):
    def closestNode(self, n, edges, query):
        """
        :type n: int
        :type edges: List[List[int]]
        :type query: List[List[int]]
        :rtype: List[int]
        """
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v), adj[v].append(u)
        tree_infos = TreeInfos3(adj)
        return [max((tree_infos.lca(x, y) for x, y in ((start, end), (start, node), (end, node))), key=lambda x: tree_infos.D[x]) for start, end, node in query]


# Time:  O(n^2 + q * n)
# Space: O(n^2)
# bfs
class Solution4(object):
    def closestNode(self, n, edges, query):
        """
        :type n: int
        :type edges: List[List[int]]
        :type query: List[List[int]]
        :rtype: List[int]
        """
        def bfs(adj, root):
            dist = [len(adj)]*len(adj)
            q = [root]
            dist[root] = 0
            d = 0
            while q:
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if d+1 >= dist[v]:
                            continue
                        dist[v] = d+1
                        new_q.append(v)
                q = new_q
                d += 1
            return dist

        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v), adj[v].append(u)
        dist = [bfs(adj, i) for i in xrange(n)]
        result = []
        for start, end, node in query:
            x = end
            while start != end:
                if dist[node][start] < dist[node][x]:
                    x = start
                start = next(u for u in adj[start] if dist[u][end] < dist[start][end])
            result.append(x)
        return result


# Time:  O(n^2 + q * n)
# Space: O(n^2)
# bfs
class Solution5(object):
    def closestNode(self, n, edges, query):
        """
        :type n: int
        :type edges: List[List[int]]
        :type query: List[List[int]]
        :rtype: List[int]
        """
        def bfs(adj, root):
            dist = [len(adj)]*len(adj)
            q = [root]
            dist[root] = 0
            d = 0
            while q:
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if d+1 >= dist[v]:
                            continue
                        dist[v] = d+1
                        new_q.append(v)
                q = new_q
                d += 1
            return dist

        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v), adj[v].append(u)
        dist = [bfs(adj, i) for i in xrange(n)]
        return [max((i for i in xrange(n) if dist[start][node]+dist[node][end]-2*dist[node][i] == dist[start][i]+dist[i][end]), key=lambda x: dist[node][x]) for start, end, node in query]

# 2281 Hard 2281 Sum of Total Strength of Wizards

In [ ]:
# Time:  O(n)
# Space: O(n)

# mono stack, prefix sum, optimized from solution2
class Solution(object):
    def totalStrength(self, strength):
        """
        :type strength: List[int]
        :rtype: int
        """
        MOD = 10**9+7
        curr = 0
        prefix = [0]*(len(strength)+1)
        for i in xrange(len(strength)):
            curr = (curr+strength[i])%MOD
            prefix[i+1] = (prefix[i]+curr)%MOD
        stk, result = [-1], 0
        for i in xrange(len(strength)+1):
            while stk[-1] != -1 and (i == len(strength) or strength[stk[-1]] >= strength[i]):
                x, y, z = stk[-2]+1, stk.pop(), i-1
                # assert(all(strength[j] >= strength[y] for j in xrange(x, y+1)))
                # assert(all(strength[j] > strength[y] for j in xrange(y+1, z+1)))
                result = (result+(strength[y]*((y-x+1)*(prefix[z+1]-prefix[y])-(z-y+1)*(prefix[y]-prefix[max(x-1, 0)]))))%MOD
            stk.append(i)
        return result


# Time:  O(n)
# Space: O(n)
# mono stack, prefix sum
class Solution2(object):
    def totalStrength(self, strength):
        """
        :type strength: List[int]
        :rtype: int
        """
        MOD = 10**9+7
        prefix, prefix2 = [0]*(len(strength)+1), [0]*(len(strength)+1)
        for i in xrange(len(strength)):
            prefix[i+1] = (prefix[i]+strength[i])%MOD
            prefix2[i+1] = (prefix2[i]+strength[i]*(i+1))%MOD
        suffix, suffix2 = [0]*(len(strength)+1), [0]*(len(strength)+1)
        for i in reversed(xrange(len(strength))):
            suffix[i] = (suffix[i+1]+strength[i])%MOD
            suffix2[i] = (suffix2[i+1]+strength[i]*(len(strength)-i))%MOD
        stk, result = [-1], 0
        for i in xrange(len(strength)+1):
            while stk[-1] != -1 and (i == len(strength) or strength[stk[-1]] >= strength[i]):
                x, y, z = stk[-2]+1, stk.pop(), i-1
                # assert(all(strength[j] >= strength[y] for j in xrange(x, y+1)))
                # assert(all(strength[j] > strength[y] for j in xrange(y+1, z+1)))
                result = (result+(strength[y]*((z-y+1)*((prefix2[y+1]-prefix2[x])-x*(prefix[y+1]-prefix[x]))+
                                               (y-x+1)*((suffix2[y+1]-suffix2[z+1])-(len(strength)-(z+1))*(suffix[y+1]-suffix[z+1])))))%MOD
            stk.append(i)
        return result

# 2286 Hard 2286 Booking Concert Tickets in Groups

In [ ]:
# Time:  ctor:    O(n)
#        gather:  O(logn)
#        scatter: O(logn), amortized
# Space: O(n)

# Template:
# https://github.com/kamyu104/LeetCode-Solutions/blob/master/Python/longest-substring-of-one-repeating-character.py
class SegmentTree(object):
    def __init__(self, N,
                 build_fn=lambda _: float("inf"),
                 query_fn=lambda x, y: y if x is None else x if y is None else min(x, y),
                 update_fn=lambda x: x):
        self.tree = [None]*(2*2**((N-1).bit_length()))
        self.base = len(self.tree)//2
        self.query_fn = query_fn
        self.update_fn = update_fn
        for i in xrange(self.base, self.base+N):
            self.tree[i] = build_fn(i-self.base)
        for i in reversed(xrange(1, self.base)):
            self.tree[i] = query_fn(self.tree[2*i], self.tree[2*i+1])

    def update(self, i, h):
        x = self.base+i
        self.tree[x] = self.update_fn(h)
        while x > 1:
            x //= 2
            self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])

    def query(self, L, R):
        L += self.base
        R += self.base
        left = right = None
        while L <= R:
            if L & 1:
                left = self.query_fn(left, self.tree[L])
                L += 1
            if R & 1 == 0:
                right = self.query_fn(self.tree[R], right)
                R -= 1
            L //= 2
            R //= 2
        return self.query_fn(left, right)


# design, segment tree, binary search
class BookMyShow(object):

    def __init__(self, n, m):
        """
        :type n: int
        :type m: int
        """
        self.__st = SegmentTree(n,
                                build_fn=lambda _: [m]*2,
                                query_fn=lambda x, y: y if x is None else x if y is None else [max(x[0], y[0]), x[1]+y[1]])
        self.__m = m
        self.__i = 0

    def gather(self, k, maxRow):
        """
        :type k: int
        :type maxRow: int
        :rtype: List[int]
        """
        i = 1
        if k > self.__st.tree[i][0]:
            return []
        while i < self.__st.base:
            i = 2*i+int(self.__st.tree[2*i][0] < k)
        if i-self.__st.base > maxRow:
            return []
        cnt = self.__st.tree[i][0]
        c = self.__m-cnt
        i -= self.__st.base
        self.__st.update(i, [cnt-k]*2)
        return [i, c]

    def scatter(self, k, maxRow):
        """
        :type k: int
        :type maxRow: int
        :rtype: bool
        """
        cnt = self.__st.query(self.__i, maxRow)
        if not cnt or cnt[1] < k:
            return False
        for i in xrange(self.__i, maxRow+1):
            cnt = self.__st.tree[self.__st.base+i][1]
            c = min(cnt, k)
            cnt -= c
            if not cnt:
                self.__i += 1
            self.__st.update(i, [cnt]*2)
            k -= c
            if not k:
                break
        return True

# 2290 Hard 2290 Minimum Obstacle Removal to Reach Corner

In [ ]:
# Time:  O(m * n)
# Space: O(m * n)

# A* Search Algorithm without heap
class Solution(object):
    def minimumObstacles(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        def a_star(grid, b, t):
            f, dh = 0, 1
            closer, detour = [b], []
            lookup = set()
            while closer or detour:
                if not closer:
                    f += dh
                    closer, detour = detour, closer
                b = closer.pop()
                if b in lookup:
                    continue
                lookup.add(b)
                if b == t:
                    return f
                for dr, dc in directions:
                    nb = (b[0]+dr, b[1]+dc)
                    if not (0 <= nb[0] < len(grid) and 0 <= nb[1] < len(grid[0]) and nb not in lookup):
                        continue
                    (closer if not grid[b[0]][b[1]] else detour).append(nb)
            return -1

        return a_star(grid, (0, 0), (len(grid)-1, len(grid[0])-1))


# Time:  O(m * n)
# Space: O(m * n)
import collections


# 0-1 bfs solution
class Solution2(object):
    def minimumObstacles(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        b, t = (0, 0), (len(grid)-1, len(grid[0])-1)
        dq = collections.deque([(b, 0)])
        lookup = set()
        while dq:
            b, d = dq.popleft()
            if b in lookup:
                continue
            lookup.add(b)
            if b == t:
                return d
            for dr, dc in directions:
                nb = (b[0]+dr, b[1]+dc)
                if not (0 <= nb[0] < len(grid) and 0 <= nb[1] < len(grid[0]) and nb not in lookup):
                    continue
                if not grid[b[0]][b[1]]:
                    dq.appendleft((nb, d))
                else:
                    dq.append((nb, d+1))
        return -1  # never reach here

# 2296 Hard 2296 Design a Text Editor

In [ ]:
# Time:  ctor:        O(1)
#        addText:     O(l)
#        deleteText:  O(k)
#        cursorLeft:  O(k)
#        cursorRight: O(k)
# Space: O(n)

# design, stack
class TextEditor(object):

    def __init__(self):
        self.__LAST_COUNT = 10
        self.__left = []
        self.__right = []

    def addText(self, text):
        """
        :type text: str
        :rtype: None
        """
        for x in text:
            self.__left.append(x)

    def deleteText(self, k):
        """
        :type k: int
        :rtype: int
        """
        return self.__move(k, self.__left, None)

    def cursorLeft(self, k):
        """
        :type k: int
        :rtype: str
        """
        self.__move(k, self.__left, self.__right)
        return self.__last_characters()

    def cursorRight(self, k):
        """
        :type k: int
        :rtype: str
        """
        self.__move(k, self.__right, self.__left)
        return self.__last_characters()

    def __move(self, k, src, dst):
        cnt = min(k, len(src))
        for _ in xrange(cnt):
            if dst is not None:
                dst.append(src[-1])
            src.pop()
        return cnt

    def __last_characters(self):
        return "".join(self.__left[-self.__LAST_COUNT:])

# 2301 Hard 2301 Match Substring After Replacement

In [ ]:
# Time:  O(n * k), n = len(s), k = len(sub)
# Space: O(m), m = len(mappings)

import collections


# brute force
class Solution(object):
    def matchReplacement(self, s, sub, mappings):
        """
        :type s: str
        :type sub: str
        :type mappings: List[List[str]]
        :rtype: bool
        """
        def transform(x):
            return ord(x)-ord('0') if x.isdigit() else ord(x)-ord('a')+10 if x.islower() else ord(x)-ord('A')+36

        def check(i):
            return all(sub[j] == s[i+j] or lookup[sub[j]][s[i+j]] for j in xrange(len(sub)))
            
        lookup = [[0]*62 for _ in xrange(62)]
        for a, b in mappings:
            lookup[transform(a)][transform(b)] = 1
        s = map(transform, s)
        sub = map(transform, sub)
        return any(check(i) for i in xrange(len(s)-len(sub)+1))


# Time:  O(n * k), n = len(s), k = len(sub)
# Space: O(m), m = len(mappings)
import collections


# brute force
class Solution2(object):
    def matchReplacement(self, s, sub, mappings):
        """
        :type s: str
        :type sub: str
        :type mappings: List[List[str]]
        :rtype: bool
        """
        def check(i):
            return all(sub[j] == s[i+j] or (sub[j], s[i+j]) in lookup for j in xrange(len(sub)))
            
        lookup = set()
        for a, b in mappings:
            lookup.add((a, b))
        return any(check(i) for i in xrange(len(s)-len(sub)+1))

# 2302 Hard 2302 Count Subarrays With Score Less Than K

In [ ]:
# Time:  O(n)
# Space: O(1)

# sliding window, two pointers
class Solution(object):
    def countSubarrays(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        result = total = left = 0
        for right in xrange(len(nums)):
            total += nums[right]
            while total*(right-left+1) >= k:
                total -= nums[left]
                left += 1
            result += right-left+1
        return result

# 2306 Hard 2306 Naming a Company

In [ ]:
# Time:  O(26 * n * l)
# Space: O(n * l)

# hash table, math
class Solution(object):
    def distinctNames(self, ideas):
        """
        :type ideas: List[str]
        :rtype: int
        """
        lookup = [set() for _ in xrange(26)]
        for x in ideas:
            lookup[ord(x[0])-ord('a')].add(x[1:])
        result = 0
        for i in xrange(len(lookup)):
            for j in xrange(i+1, len(lookup)):
                common = len(lookup[i]&lookup[j])
                result += (len(lookup[i])-common)*(len(lookup[j])-common)
        return result*2

# 2307 Hard 2307 Check for Contradictions in Equations

In [ ]:
# Time:  O(e + q)
# Space: O(n)

import collections
import itertools


class UnionFind(object):
    def __init__(self):
        self.set = {}
        self.rank = collections.Counter()

    def find_set(self, x):
        xp, xr = self.set.setdefault(x, (x, 1.0))
        if x != xp:
            pp, pr = self.find_set(xp)  # path compression.
            self.set[x] = (pp, xr*pr)  # x/pp = xr*pr
        return self.set[x]

    def union_set(self, x, y, r):
        (xp, xr), (yp, yr) =  map(self.find_set, (x, y))
        if xp == yp:
            return False
        if self.rank[xp] < self.rank[yp]:  # union by rank
            # to make x/yp = r*yr and merge xp into yp
            # => since x/xp = xr, we can merge with xp/yp = r*yr/xr 
            self.set[xp] = (yp, r*yr/xr)
        elif self.rank[xp] > self.rank[yp]:
            # to make y/xp = 1/r*xr and merge xp into yp
            # => since y/yp = yr, we can merge with yp/xp = 1/r*xr/yr 
            self.set[yp] = (xp, 1.0/r*xr/yr)
        else:
            # to make y/xp = 1/r*xr and merge xp into yp
            # => since y/yp = yr, we can merge with yp/xp = 1/r*xr/yr 
            self.set[yp] = (xp, 1.0/r*xr/yr)
            self.rank[xp] += 1 
        return True

    def query_set(self, x, y):
        if x not in self.set or y not in self.set:
            return -1.0
        (xp, xr), (yp, yr) = map(self.find_set, (x, y))
        return xr/yr if xp == yp else -1.0


# Time:  O(e + q)
# Space: O(n)
import itertools


# union find
class Solution(object):
    def checkContradictions(self, equations, values):
        """
        :type equations: List[List[str]]
        :type values: List[float]
        :rtype: bool
        """
        EPS = 1e-5
        uf = UnionFind()
        return any(not uf.union_set(a, b, k) and abs(uf.query_set(a, b)-k) >= EPS for (a, b), k in itertools.izip(equations, values))


# Time:  O(e + q)
# Space: O(n)
import collections
import itertools


# dfs
class Solution2(object):
    def checkContradictions(self, equations, values):
        """
        :type equations: List[List[str]]
        :type values: List[float]
        :rtype: bool
        """
        def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
            return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

        def iter_dfs(adj, u, lookup):
            stk = [u]
            lookup[u] = 1.0
            while stk:
                u = stk.pop()
                for v, k in adj[u]:
                    if v in lookup:
                        if not isclose(lookup[v], lookup[u]*k):
                            return True
                        continue
                    lookup[v] = lookup[u]*k
                    stk.append(v)
            return False

        adj = collections.defaultdict(set)
        for (a, b), k in itertools.izip(equations, values):
            adj[a].add((b, 1.0/k))
            adj[b].add((a, 1.0*k))
        lookup = {}
        for u in adj.iterkeys():
            if u in lookup:
                continue
            if iter_dfs(adj, u, lookup):
                return True
        return False

# 2312 Hard 2312 Selling Pieces of Wood

In [ ]:
# Time:  O(m * n * (m + n))
# Space: O(m * n)

# dp
class Solution(object):
    def sellingWood(self, m, n, prices):
        """
        :type m: int
        :type n: int
        :type prices: List[List[int]]
        :rtype: int
        """
        dp = [[0]*(n+1) for i in xrange(m+1)]
        for h, w, p in prices:
            dp[h][w] = p
        for i in xrange(1, m+1):
            for j in xrange(1, n+1):
                for k in xrange(1, i//2+1):
                    dp[i][j] = max(dp[i][j], dp[k][j]+dp[i-k][j])
                for k in xrange(1, j//2+1):
                    dp[i][j] = max(dp[i][j], dp[i][k]+dp[i][j-k])
        return dp[m][n]

# 2313 Hard 2313 Minimum Flips in Binary Tree to Get Result

In [ ]:
# Time:  O(n)
# Space: O(h)

class TreeNode(object):
    def __init__(self, val=0, left=None, right=None):
        pass


import collections


# tree dp with stack
class Solution(object):
    def minimumFlips(self, root, result):
        """
        :type root: Optional[TreeNode]
        :type result: bool
        :rtype: int
        """
        INF = float("inf")
        OP = {
            2: lambda x, y: x or y,
            3: lambda x, y: x and y,
            4: lambda x, y: x^y ,
            5: lambda x, y: not x if x is not None else not y
        }
        
        def iter_dfs(root, result):
            ret = collections.defaultdict(lambda: INF)
            stk = [(1, (root, ret))]
            while stk:
                step, args = stk.pop()
                if step == 1:
                    node, ret = args
                    if not node:
                        ret[None] = 0 # null object pattern
                        continue
                    if node.left == node.right:
                        ret[True] = node.val^1
                        ret[False] = node.val^0
                        continue
                    ret1 = collections.defaultdict(lambda: INF)
                    ret2 = collections.defaultdict(lambda: INF)
                    stk.append((2, (node, ret1, ret2, ret)))
                    stk.append((1, (node.right, ret2)))
                    stk.append((1, (node.left, ret1)))
                elif step == 2:
                    node, ret1, ret2, ret = args
                    for k1, v1 in ret1.iteritems():
                        for k2, v2 in ret2.iteritems():
                            ret[OP[node.val](k1, k2)] = min(ret[OP[node.val](k1, k2)], v1+v2)
            return ret[result]

        return iter_dfs(root, result)


import collections


# tree dp with recursion
class Solution2(object):
    def minimumFlips(self, root, result):
        """
        :type root: Optional[TreeNode]
        :type result: bool
        :rtype: int
        """
        INF = float("inf")
        OP = {
            2: lambda x, y: x or y,
            3: lambda x, y: x and y,
            4: lambda x, y: x^y ,
            5: lambda x, y: not x if x is not None else not y
        }
        
        def dfs(node):
            if not node:
                return {None: 0}  # null object pattern
            if node.left == node.right:
                return {True: node.val^1, False: node.val^0}
            left = dfs(node.left)
            right = dfs(node.right)
            dp = collections.defaultdict(lambda: INF)
            for k1, v1 in left.iteritems():
                for k2, v2 in right.iteritems():
                    dp[OP[node.val](k1, k2)] = min(dp[OP[node.val](k1, k2)], v1+v2)
            return dp

        return dfs(root)[result]

# 2318 Hard 2318 Number of Distinct Roll Sequences

In [ ]:
# Time:  O(6^3 * n)
# Space: O(6^2)

import collections


# dp
class Solution(object):
    def distinctSequences(self, n):
        """
        :type n: int
        :rtype: int
        """
        def gcd(a, b):
            while b:
                a, b = b, a%b
            return a

        if n == 1:
            return 6
        MOD = 10**9 + 7
        dp = [[0]*6 for _ in xrange(6)]
        for i in xrange(6):
            for j in xrange(6):
                if i != j and gcd(i+1, j+1) == 1:
                    dp[i][j] = 1
        for _ in xrange(n-2):
            new_dp = [[0]*6 for _ in xrange(6)]
            for i in xrange(6):
                for j in xrange(6):
                    if not dp[i][j]:
                        continue
                    for k in xrange(6):
                        if not dp[j][k]:
                            continue
                        if k != i:
                            new_dp[i][j] = (new_dp[i][j]+dp[j][k]) % MOD
            dp = new_dp
        return sum(dp[i][j] for i in xrange(6) for j in xrange(6)) % MOD

# 2321 Hard 2321 Maximum Score Of Spliced Array

In [ ]:
# Time:  O(n)
# Space: O(1)

# greedy, kadane's algorithm
class Solution(object):
    def maximumsSplicedArray(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: int
        """
        def kadane(a):
            result = curr = 0
            for x in a:
                curr = max(curr+x, 0)
                result = max(result, curr)
            return result
    
        return max(sum(nums1)+kadane((nums2[i]-nums1[i] for i in xrange(len(nums1)))),
                   sum(nums2)+kadane((nums1[i]-nums2[i] for i in xrange(len(nums2)))))

# 2322 Hard 2322 Minimum Score After Removals on a Tree

In [ ]:
# Time:  O(n^2)
# Space: O(n)

# dfs with stack
class Solution(object):
    def minimumScore(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def is_ancestor(a, b):
            return left[a] <= left[b] and right[b] <= right[a]

        def iter_dfs():
            cnt = 0
            left = [0]*len(nums)
            right = [0]*len(nums)
            stk = [(1, (0, -1))]
            while stk:
                step, args = stk.pop()
                if step == 1:
                    u, p = args
                    left[u] = cnt
                    cnt += 1
                    stk.append((2, (u, p)))
                    for v in adj[u]:
                        if v == p:
                            continue
                        stk.append((1, (v, u)))
                elif step == 2:
                    u, p = args
                    for v in adj[u]:
                        if v == p:
                            continue
                        nums[u] ^= nums[v]
                    right[u] = cnt
            return left, right
                
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        left, right = iter_dfs()
        result = float("inf")
        for i in xrange(1, len(nums)):
            for j in xrange(i+1, len(nums)):
                if is_ancestor(i, j):
                    a, b, c = nums[0]^nums[i], nums[i]^nums[j], nums[j]
                elif is_ancestor(j, i):
                    a, b, c = nums[0]^nums[j], nums[j]^nums[i], nums[i]
                else:
                    a, b, c = nums[0]^nums[i]^nums[j], nums[i], nums[j]
                result = min(result, max(a, b, c)-min(a, b, c))
        return result


# Time:  O(n^2)
# Space: O(n)
# dfs with recursion
class Solution2(object):
    def minimumScore(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def is_ancestor(a, b):
            return left[a] <= left[b] and right[b] <= right[a]

        def dfs(u, p):
            left[u] = cnt[0]
            cnt[0] += 1
            for v in adj[u]:
                if v == p:
                    continue
                dfs(v, u)
                nums[u] ^= nums[v]
            right[u] = cnt[0]
                
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        cnt = [0]
        left = [0]*len(nums)
        right = [0]*len(nums)
        dfs(0, -1)
        result = float("inf")
        for i in xrange(1, len(nums)):
            for j in xrange(i+1, len(nums)):
                if is_ancestor(i, j):
                    a, b, c = nums[0]^nums[i], nums[i]^nums[j], nums[j]
                elif is_ancestor(j, i):
                    a, b, c = nums[0]^nums[j], nums[j]^nums[i], nums[i]
                else:
                    a, b, c = nums[0]^nums[i]^nums[j], nums[i], nums[j]
                result = min(result, max(a, b, c)-min(a, b, c))
        return result
            

# Time:  O(n^2)
# Space: O(n)
# dfs with recursion
class Solution3(object):
    def minimumScore(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def dfs(u, p, result):
            total = nums[u]
            for v in adj[u]:
                if v == p:
                    continue
                total ^= dfs(v, u, result)
            result.append(total)
            return total
                
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        total = reduce(lambda x, y: x^y, nums)
        result = float("inf")
        for u, v in edges: 
            left = []
            dfs(u, v, left)
            right = []
            dfs(v, u, right)
            for candidates in (left, right):
                total2 = candidates.pop()
                for x in candidates:
                    a, b, c = total^total2, x, total2^x
                    result = min(result, max(a, b, c)-min(a, b, c))
        return result
            

# Time:  O(n^2)
# Space: O(n)
# dfs with stk (slower, sometimes TLE)
class Solution4(object):
    def minimumScore(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def iter_dfs(nums, adj, u, p):
            result = []
            stk = [(1, (u, p, [0]))]
            while stk:
                step, args = stk.pop()
                if step == 1:
                    u, p, ret = args
                    new_rets = []
                    stk.append((2, (u, new_rets, ret)))
                    for v in adj[u]:
                        if v == p:
                            continue
                        new_rets.append([0])
                        stk.append((1, (v, u, new_rets[-1])))
                elif step == 2:
                    u, new_rets, ret = args
                    ret[0] = nums[u]
                    for x in new_rets:
                        ret[0] ^= x[0]
                    result.append(ret[0])
            return result
                
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        total = reduce(lambda x, y: x^y, nums)
        result = float("inf")
        for u, v in edges: 
            for candidates in (iter_dfs(nums, adj, u, v), iter_dfs(nums, adj, v, u)):
                total2 = candidates.pop()
                for x in candidates:
                    a, b, c = total^total2, x, total2^x
                    result = min(result, max(a, b, c)-min(a, b, c))
        return result

# 2328 Hard 2328 Number of Increasing Paths in a Grid

In [ ]:
# Time:  O(m * n)
# Space: O(m * n)

# topological sort, bottom-up dp
class Solution(object):
    def countPaths(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        MOD = 10**9+7
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        in_degree = [[0]*len(grid[0]) for _ in xrange(len(grid))]
        q = []
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                for di, dj in directions:
                    ni, nj = i+di, j+dj
                    if 0 <= ni < len(grid) and 0 <= nj < len(grid[0]) and grid[i][j] > grid[ni][nj]:
                        in_degree[i][j] += 1
                if not in_degree[i][j]:
                    q.append((i, j))
        dp = [[1]*len(grid[0]) for _ in xrange(len(grid))]
        result = 0
        while q:
            new_q = []
            for i, j in q:
                result = (result+dp[i][j])%MOD
                for di, dj in directions:
                    ni, nj = i+di, j+dj
                    if not (0 <= ni < len(grid) and 0 <= nj < len(grid[0]) and grid[i][j] < grid[ni][nj]):
                        continue
                    dp[ni][nj] = (dp[ni][nj]+dp[i][j])%MOD
                    in_degree[ni][nj] -= 1
                    if not in_degree[ni][nj]:
                        new_q.append((ni, nj))
            q = new_q
        return result


# Time:  O(m * n)
# Space: O(m * n)
# top-down dp, memoization
class Solution2(object):
    def countPaths(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        MOD = 10**9+7
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        def memoization(grid, i, j, lookup):
            if not lookup[i][j]:
                lookup[i][j] = 1
                for di, dj in directions:
                    ni, nj = i+di, j+dj
                    if 0 <= ni < len(grid) and 0 <= nj < len(grid[0]) and grid[i][j] < grid[ni][nj]:
                        lookup[i][j] = (lookup[i][j]+memoization(grid, ni, nj, lookup)) % MOD
            return lookup[i][j]

        lookup = [[0]*len(grid[0]) for _ in xrange(len(grid))]
        return sum(memoization(grid, i, j, lookup) for i in xrange(len(grid)) for j in xrange(len(grid[0]))) % MOD

# 2334 Hard 2334 Subarray With Elements Greater Than Varying Threshold

In [ ]:
# Time:  O(n)
# Space: O(n)

# mono stack
class Solution(object):
    def validSubarraySize(self, nums, threshold):
        """
        :type nums: List[int]
        :type threshold: int
        :rtype: int
        """
        stk = [-1]
        for i in xrange(len(nums)+1):
            while stk[-1] != -1 and (i == len(nums) or nums[stk[-1]] >= nums[i]):
                if nums[stk.pop()]*((i-1)-stk[-1]) > threshold:
                    return (i-1)-stk[-1]
            stk.append(i)
        return -1

# 2338 Hard 2338 Count the Number of Ideal Arrays

In [ ]:
# Time:  O(sqrt(m) + n + m * (logm + sqrt(m)/log(sqrt(m))))
# Space: O(sqrt(m) + n + logm)

import collections


# dp, factorization, combinatorics
class Solution(object):
    def idealArrays(self, n, maxValue):
        """
        :type n: int
        :type maxValue: int
        :rtype: int
        """
        MOD = 10**9+7
        fact, inv, inv_fact = [[1]*2 for _ in xrange(3)]
        def nCr(n, k):
            while len(inv) <= n:  # lazy initialization
                fact.append(fact[-1]*len(inv) % MOD)
                inv.append(inv[MOD%len(inv)]*(MOD-MOD//len(inv)) % MOD)  # https://cp-algorithms.com/algebra/module-inverse.html
                inv_fact.append(inv_fact[-1]*inv[-1] % MOD)
            return (fact[n]*inv_fact[n-k] % MOD) * inv_fact[k] % MOD

        def linear_sieve_of_eratosthenes(n):  # Time: O(n), Space: O(n)
            primes = []
            spf = [-1]*(n+1)  # the smallest prime factor
            for i in xrange(2, n+1):
                if spf[i] == -1:
                    spf[i] = i
                    primes.append(i)
                for p in primes:
                    if i*p > n or p > spf[i]:
                        break
                    spf[i*p] = p
            return primes

        def prime_factors(x):
            factors = collections.Counter()
            for p in primes:
                if x < p:
                    break
                while x%p == 0:
                    factors[p] += 1
                    x //= p
            if x != 1:
                factors[x] += 1
            return factors

        primes = linear_sieve_of_eratosthenes(int(maxValue**0.5))
        result = 0
        for k in xrange(1, maxValue+1):
            total = 1
            for c in prime_factors(k).itervalues():
                total = (total*nCr(n+c-1, c))%MOD  # H(n, c) = nCr(n+c-1, n)
            result = (result+total)%MOD
        return result


# Time:  O(n * mlogm)
# Space: O(n + m)
import collections


# dp, combinatorics
class Solution2(object):
    def idealArrays(self, n, maxValue):
        """
        :type n: int
        :type maxValue: int
        :rtype: int
        """
        MOD = 10**9+7
        fact, inv, inv_fact = [[1]*2 for _ in xrange(3)]
        def nCr(n, k):
            while len(inv) <= n:  # lazy initialization
                fact.append(fact[-1]*len(inv) % MOD)
                inv.append(inv[MOD%len(inv)]*(MOD-MOD//len(inv)) % MOD)  # https://cp-algorithms.com/algebra/module-inverse.html
                inv_fact.append(inv_fact[-1]*inv[-1] % MOD)
            return (fact[n]*inv_fact[n-k] % MOD) * inv_fact[k] % MOD

        result = 0
        dp = collections.Counter(xrange(1, maxValue+1))
        for i in xrange(n): 
            new_dp = collections.Counter()
            total = 0
            for x, c in dp.iteritems():
                total = (total+c)%MOD
                for y in xrange(x+x, maxValue+1, x): 
                    new_dp[y] += c
            result = (result+total*nCr(n-1, i))%MOD
            dp = new_dp
        return result

# 2344 Hard 2344 Minimum Deletions to Make Array Divisible

In [ ]:
# Time:  O(n + m + logr), r is max(numsDivide)
# Space: O(1)

# gcd
class Solution(object):
    def minOperations(self, nums, numsDivide):
        """
        :type nums: List[int]
        :type numsDivide: List[int]
        :rtype: int
        """
        def gcd(a, b):  # Time: O(log(min(a, b)))
            while b:
                a, b = b, a%b
            return a

        g = reduce(gcd, numsDivide)
        mn = float("inf")
        for x in nums:
            if g%x == 0:
                mn = min(mn, x)
        return sum(x < mn for x in nums) if mn != float("inf") else -1

# 2350 Hard 2350 Shortest Impossible Sequence of Rolls

In [ ]:
# Time:  O(n)
# Space: O(k)

# constructive algorithms
class Solution(object):
    def shortestSequence(self, rolls, k):
        """
        :type rolls: List[int]
        :type k: int
        :rtype: int
        """
        l = 0
        lookup = set()
        for x in rolls:
            lookup.add(x)
            if len(lookup) != k:
                continue
            lookup.clear()
            l += 1
        return l+1

# 2354 Hard 2354 Number of Excellent Pairs

In [ ]:
# Time:  O(n + (logn)^2) = O(n)
# Space: O(n + logn) = O(n)

import collections


# bit manipulation, freq table, combinatorics
class Solution(object):
    def countExcellentPairs(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def popcount(x):
            return bin(x)[2:].count('1')

        cnt = collections.Counter(popcount(x) for x in set(nums))
        return sum(cnt[i]*cnt[j] for i in cnt.iterkeys() for j in cnt.iterkeys() if i+j >= k)


# Time:  O(nlogn)
# Space: O(n)
# bit manipulation, sort, two pointers
class Solution2(object):
    def countExcellentPairs(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def popcount(x):
            return bin(x)[2:].count('1')

        sorted_cnts = sorted(popcount(x) for x in set(nums))
        result = 0
        left, right = 0, len(sorted_cnts)-1
        while left <= right:
            if sorted_cnts[left]+sorted_cnts[right] < k:
                left += 1
            else:
                result += 1+2*((right-1)-left+1)
                right -= 1
        return result

# 2355 Hard 2355 Maximum Number of Books You Can Take

In [ ]:
# Time:  O(n)
# Space: O(n)

# mono stack
class Solution(object):
    def maximumBooks(self, books):
        """
        :type books: List[int]
        :rtype: int
        """
        def count(right, l):
            left = max(right-l+1, 0)
            return (left+right)*(right-left+1)//2
        
        result = curr = 0
        stk = [-1]
        for i in xrange(len(books)):
            while stk[-1] != -1 and books[stk[-1]] >= books[i]-(i-stk[-1]):
                j = stk.pop()
                curr -= count(books[j], j-stk[-1])
            curr += count(books[i], i-stk[-1])
            stk.append(i)
            result = max(result, curr)
        return result

# 2360 Hard 2360 Longest Cycle in a Graph

In [ ]:
# Time:  O(n)
# Space: O(n)

# graph
class Solution(object):
    def longestCycle(self, edges):
        """
        :type edges: List[int]
        :rtype: int
        """
        result = -1
        lookup = [-1]*len(edges)
        idx = 0
        for i in xrange(len(edges)):
            if lookup[i] != -1:
                continue
            start = idx
            while i != -1:
                if lookup[i] != -1:
                    break
                lookup[i] = idx
                idx += 1
                i = edges[i]
            if i != -1 and lookup[i] >= start:
                result = max(result, idx-lookup[i])
        return result

# 2361 Hard 2361 Minimum Costs Using the Train Line

In [ ]:
# Time:  O(n)
# Space: O(1)

import itertools


# dp
class Solution(object):
    def minimumCosts(self, regular, express, expressCost):
        """
        :type regular: List[int]
        :type express: List[int]
        :type expressCost: int
        :rtype: List[int]
        """
        result = []
        dp = [0, expressCost]  # dp[0]: min cost of regular route to curr stop, dp[1]: min cost of express route to curr stop
        for r, e in itertools.izip(regular, express):
            dp = [min(dp[0]+r, dp[1]+e), min(dp[0]+(r+expressCost), dp[1]+e)]
            result.append(min(dp[0], dp[1]))
        return result

# 2366 Hard 2366 Minimum Replacements to Sort the Array

In [ ]:
# Time:  O(n)
# Space: O(1)

# greedy, math
class Solution(object):
    def minimumReplacement(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def ceil_divide(a, b):
            return (a+b-1)//b

        result = 0
        curr = nums[-1]
        for x in reversed(nums):
            cnt = ceil_divide(x, curr)
            result += cnt-1
            curr = x//cnt
        return result

# 2371 Hard 2371 Minimize Maximum Value in a Grid

In [ ]:
# Time:  O((m * n) * log(m * n))
# Space: O(m * n)

# sort, greedy
class Solution(object):
    def minScore(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: List[List[int]]
        """
        idxs = [(i, j) for i in xrange(len(grid)) for j in xrange(len(grid[0]))]
        idxs.sort(key=lambda x: grid[x[0]][x[1]])
        row_max, col_max = [0]*len(grid), [0]*len(grid[0])
        for i, j in idxs:
            grid[i][j] = row_max[i] = col_max[j] = max(row_max[i], col_max[j])+1
        return grid

# 2376 Hard 2376 Count Special Integers

In [ ]:
# Time:  O(logn)
# Space: O(logn)

# combinatorics
class Solution(object):
    def countSpecialNumbers(self, n):
        """
        :type n: int
        :rtype: int
        """
        def P(m, n):
            result = 1
            for _ in xrange(n):
                result *= m
                m -= 1
            return result

        digits = map(int, str(n+1))
        result = sum(P(9, 1)*P(9, i-1) for i in xrange(1, len(digits)))
        lookup = set()
        for i, x in enumerate(digits):
            for y in xrange(int(i == 0), x):
                if y in lookup:
                    continue
                result += P(9-i, len(digits)-i-1)
            if x in lookup:
                break
            lookup.add(x)
        return result

# 2382 Hard 2382 Maximum Segment Sum After Removals

In [ ]:
# Time:  O(n)
# Space: O(n)

class UnionFind(object):  # Time: O(n * alpha(n)), Space: O(n)
    def __init__(self, nums):
        self.set = range(len(nums))
        self.rank = [0]*len(nums)
        self.size = nums[:]

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x, y = self.find_set(x), self.find_set(y)
        if x == y:
            return False
        if self.rank[x] > self.rank[y]:  # union by rank
            x, y = y, x
        self.set[x] = self.set[y]
        if self.rank[x] == self.rank[y]:
            self.rank[y] += 1
        self.size[y] += self.size[x]
        return True

    def total(self, x):
        return self.size[self.find_set(x)]


# union find
class Solution(object):
    def maximumSegmentSum(self, nums, removeQueries):
        """
        :type nums: List[int]
        :type removeQueries: List[int]
        :rtype: List[int]
        """
        result = [0]*len(removeQueries)
        lookup = [0]*len(nums)
        uf = UnionFind(nums)
        for i in reversed(xrange(1, len(removeQueries))): 
            q = removeQueries[i]
            lookup[q] = 1
            if q-1 >= 0 and lookup[q-1]:
                uf.union_set(q-1, q)
            if q+1 < len(nums) and lookup[q+1]:
                uf.union_set(q, q+1)
            result[i-1] = max(result[i], uf.total(q))   
        return result


# Time:  O(nlogn)
# Space: O(n)
from sortedcontainers import SortedList


# prefix sum, sorted list
class Solution2(object):
    def maximumSegmentSum(self, nums, removeQueries):
        """
        :type nums: List[int]
        :type removeQueries: List[int]
        :rtype: List[int]
        """
        removed_idxs = SortedList([-1, len(nums)])
        prefix = [0]*(len(nums)+1)
        for i in xrange(len(nums)):
            prefix[i+1] = prefix[i]+nums[i]
        segments = SortedList([prefix[-1]])
        result = []
        for q in removeQueries: 
            removed_idxs.add(q)
            i = removed_idxs.bisect_left(q)
            left, right = removed_idxs[i-1], removed_idxs[i+1]
            segments.remove(prefix[right]-prefix[left+1])
            segments.add(prefix[q]-prefix[left+1])
            segments.add(prefix[right]-prefix[q+1])
            result.append(segments[-1])
        return result

# 2392 Hard 2392 Build a Matrix With Conditions

In [ ]:
# Time:  O(k^2 + r + c), r = len(rowConditions), c = len(colConditions)
# Space: O(k + r + c)

# topological sort
class Solution(object):
    def buildMatrix(self, k, rowConditions, colConditions):
        """
        :type k: int
        :type rowConditions: List[List[int]]
        :type colConditions: List[List[int]]
        :rtype: List[List[int]]
        """
        def topological_sort(conditions):
            adj = [[] for _ in xrange(k)]
            in_degree = [0]*k
            for u, v in conditions:
                u -= 1
                v -= 1
                adj[u].append(v)
                in_degree[v] += 1
            result = []
            q = [u for u in xrange(k) if not in_degree[u]]
            while q:
                new_q = []
                for u in q:
                    result.append(u)
                    for v in adj[u]:
                        in_degree[v] -= 1
                        if in_degree[v]:
                            continue
                        new_q.append(v)
                q = new_q
            return result

        row_order = topological_sort(rowConditions)
        if len(row_order) != k:
            return []
        col_order = topological_sort(colConditions)
        if len(col_order) != k:
            return []
        row_idx = {x:i for i, x in enumerate(row_order)}
        col_idx = {x:i for i, x in enumerate(col_order)}
        result = [[0]*k for _ in xrange(k)]
        for i in xrange(k):
            result[row_idx[i]][col_idx[i]] = i+1
        return result

# 2398 Hard 2398 Maximum Number of Robots Within Budget

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


# sliding window, two pointers, mono deque
class Solution(object):
    def maximumRobots(self, chargeTimes, runningCosts, budget):
        """
        :type chargeTimes: List[int]
        :type runningCosts: List[int]
        :type budget: int
        :rtype: int
        """
        result = left = curr = 0
        dq = collections.deque()
        for right in xrange(len(chargeTimes)):
            while dq and chargeTimes[dq[-1]] <= chargeTimes[right]:
                dq.pop()
            dq.append(right)
            curr += runningCosts[right]
            if chargeTimes[dq[0]]+(right-left+1)*curr > budget:
                if dq[0] == left:
                    dq.popleft()
                curr -= runningCosts[left]
                left += 1
        return right-left+1


# Time:  O(n)
# Space: O(n)
import collections


# sliding window, two pointers, mono deque
class Solution2(object):
    def maximumRobots(self, chargeTimes, runningCosts, budget):
        """
        :type chargeTimes: List[int]
        :type runningCosts: List[int]
        :type budget: int
        :rtype: int
        """
        result = left = curr = 0
        dq = collections.deque()
        for right in xrange(len(chargeTimes)):
            while dq and chargeTimes[dq[-1]] <= chargeTimes[right]:
                dq.pop()
            dq.append(right)
            curr += runningCosts[right]
            while dq and chargeTimes[dq[0]]+(right-left+1)*curr > budget:
                if dq[0] == left:
                    dq.popleft()
                curr -= runningCosts[left]
                left += 1
            result = max(result, right-left+1)            
        return result

# 2402 Hard 2402 Meeting Rooms III

In [ ]:
# Time:  O(mlogm + n + mlogn)
# Space: O(n)

import heapq


# one heap solution
class Solution(object):
    def mostBooked(self, n, meetings):
        """
        :type n: int
        :type meetings: List[List[int]]
        :rtype: int
        """
        meetings.sort()
        min_heap = [(meetings[0][0], i) for i in xrange(n)]
        result = [0]*n
        for s, e in meetings:
            while min_heap and min_heap[0][0] < s:
                _, i = heapq.heappop(min_heap)
                heapq.heappush(min_heap, (s, i))
            e2, i = heapq.heappop(min_heap)
            heapq.heappush(min_heap, (e2+(e-s), i))
            result[i] += 1
        return max(xrange(n), key=lambda x:result[x])


# Time:  O(mlogm + n + mlogn)
# Space: O(n)
import heapq


# two heaps solution
class Solution2(object):
    def mostBooked(self, n, meetings):
        """
        :type n: int
        :type meetings: List[List[int]]
        :rtype: 
        """
        meetings.sort()
        unused, used = range(n), []
        result = [0]*n
        for s, e in meetings:
            while used and used[0][0] <= s:
                _, i = heapq.heappop(used)
                heapq.heappush(unused, i)
            if unused:
                i = heapq.heappop(unused)
                heapq.heappush(used, (e, i))
            else:
                e2, i = heapq.heappop(used)
                heapq.heappush(used, (e2+(e-s), i))
            result[i] += 1
        return max(xrange(n), key=lambda x:result[x])

# 2403 Hard 2403 Minimum Time to Kill All Monsters

In [ ]:
# Time:  O(n * 2^n)
# Space: O(2^n)

# bitmasks, dp
class Solution(object):
    def minimumTime(self, power):
        """
        :type power: List[int]
        :rtype: int
        """
        def ceil_divide(a, b):
            return (a+b-1)//b

        INF = float("inf")
        dp = {0:0}
        for gain in xrange(1, len(power)+1):
            new_dp = collections.defaultdict(lambda:INF)
            for mask in dp.iterkeys():
                for i in xrange(len(power)):
                    if mask&(1<<i) == 0:
                        new_dp[mask|(1<<i)] = min(new_dp[mask|(1<<i)], dp[mask]+ceil_divide(power[i], gain))
            dp = new_dp
        return dp[(1<<len(power))-1]

# 2407 Hard 2407 Longest Increasing Subsequence II

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import bisect


# Range Maximum Query
class SegmentTree(object):
    def __init__(self, N,
                 build_fn=lambda _: 0,
                 query_fn=lambda x, y: y if x is None else x if y is None else max(x, y),
                 update_fn=lambda x: x):
        self.tree = [None]*(2*2**((N-1).bit_length()))
        self.base = len(self.tree)//2
        self.query_fn = query_fn
        self.update_fn = update_fn
        for i in xrange(self.base, self.base+N):
            self.tree[i] = build_fn(i-self.base)
        for i in reversed(xrange(1, self.base)):
            self.tree[i] = query_fn(self.tree[2*i], self.tree[2*i+1])

    def update(self, i, h):
        x = self.base+i
        self.tree[x] = self.update_fn(h)
        while x > 1:
            x //= 2
            self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])

    def query(self, L, R):
        if L > R:
            return 0
        L += self.base
        R += self.base
        left = right = None
        while L <= R:
            if L & 1:
                left = self.query_fn(left, self.tree[L])
                L += 1
            if R & 1 == 0:
                right = self.query_fn(self.tree[R], right)
                R -= 1
            L //= 2
            R //= 2
        return self.query_fn(left, right)


# segment tree with coordinate compression
class Solution(object):
    def lengthOfLIS(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        sorted_nums = sorted({x-1 for x in nums})
        num_to_idx = {x:i for i, x in enumerate(sorted_nums)}
        st = SegmentTree(len(num_to_idx))
        for x in nums:
            x -= 1
            st.update(num_to_idx[x], st.query(bisect.bisect_left(sorted_nums, x-k), num_to_idx[x]-1)+1)
        return st.tree[1]  # st.query(0, len(num_to_idx)-1)

# 2412 Hard 2412 Minimum Money Required Before Transactions

In [ ]:
# Time:  O(n)
# Space: O(1)

# greedy, constructive algorithms
class Solution(object):
    def minimumMoney(self, transactions):
        """
        :type transactions: List[List[int]]
        :rtype: int
        """
        return sum(max(a-b, 0) for a, b in transactions)+max(a-max(a-b, 0) for a, b in transactions)  # a-max(a-b, 0) = min(a, b)

# 2416 Hard 2416 Sum of Prefix Scores of Strings

In [ ]:
# Time:  O(n * l), n is the number of words, l is the max length of words
# Space: O(t), t is the size of trie
    
import collections


# trie
class Solution(object):
    def sumPrefixScores(self, words):
        """
        :type words: List[str]
        :rtype: List[int]
        """
        _trie = lambda: collections.defaultdict(_trie)
        trie = _trie()
        for w in words:
            curr = trie
            for c in w:
                curr = curr[c]
                curr["_cnt"] = curr["_cnt"]+1 if "_cnt" in curr else 1
        result = []
        for w in words:
            cnt = 0
            curr = trie
            for c in w:
                curr = curr[c]
                cnt += curr["_cnt"]
            result.append(cnt)
        return result

# 2421 Hard 2421 Number of Good Paths

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import collections


class UnionFind(object):  # Time: O(n * alpha(n)), Space: O(n)
    def __init__(self, vals):
        self.set = range(len(vals))
        self.rank = [0]*len(vals)
        self.cnt = [collections.Counter({v:1}) for v in vals]   # added

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y, v):  # modified
        x, y = self.find_set(x), self.find_set(y)
        if x == y:
            return 0  # modified
        if self.rank[x] > self.rank[y]:  # union by rank
            x, y = y, x
        self.set[x] = self.set[y]
        if self.rank[x] == self.rank[y]:
            self.rank[y] += 1
        cx, cy = self.cnt[x][v], self.cnt[y][v]  # added
        self.cnt[y] = collections.Counter({v:cx+cy})  # added
        return cx*cy  # modified


# tree, sort, union find
class Solution(object):
    def numberOfGoodPaths(self, vals, edges):
        """
        :type vals: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        edges.sort(key=lambda x: max(vals[x[0]], vals[x[1]]))
        uf = UnionFind(vals)
        return len(vals)+sum(uf.union_set(i, j, max(vals[i], vals[j])) for i, j in edges)

# 2426 Hard 2426 Number of Pairs Satisfying Inequality

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

from sortedcontainers import SortedList
import itertools


# sorted list, binary search
class Solution(object):
    def numberOfPairs(self, nums1, nums2, diff):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :type diff: int
        :rtype: int
        """
        sl = SortedList()
        result = 0
        for x, y in itertools.izip(nums1, nums2):
            result += sl.bisect_right((x-y)+diff)
            sl.add(x-y)
        return result

    
# Time:  O(nlogn)
# Space: O(n)
import itertools
import bisect


class BIT(object):  # 0-indexed.
    def __init__(self, n):
        self.__bit = [0]*(n+1)  # Extra one for dummy node.

    def add(self, i, val):
        i += 1  # Extra one for dummy node.
        while i < len(self.__bit):
            self.__bit[i] += val
            i += (i & -i)

    def query(self, i):
        i += 1  # Extra one for dummy node.
        ret = 0
        while i > 0:
            ret += self.__bit[i]
            i -= (i & -i)
        return ret


# bit, fenwick tree, coordinate compression
class Solution2(object):
    def numberOfPairs(self, nums1, nums2, diff):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :type diff: int
        :rtype: int
        """
        sorted_nums = sorted(set(x-y for x, y in itertools.izip(nums1, nums2)))
        num_to_idx = {x:i for i, x in enumerate(sorted_nums)}
        result = 0
        bit = BIT(len(num_to_idx))
        for x, y in itertools.izip(nums1, nums2):
            result += bit.query(bisect.bisect_right(sorted_nums, (x-y)+diff)-1)
            bit.add(num_to_idx[x-y], 1)
        return result


# Time:  O(nlogn)
# Space: O(n)
import itertools


# merge sort, two pointers
class Solution3(object):
    def numberOfPairs(self, nums1, nums2, diff):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :type diff: int
        :rtype: int
        """
        def merge_sort(nums, left, right, result):
            if left == right:
                return
            mid = left+(right-left)//2
            merge_sort(nums, left, mid, result)
            merge_sort(nums, mid+1, right, result)
            r = mid+1
            for l in xrange(left, mid+1):
                while r < right+1 and nums[l]-nums[r] > diff:
                    r += 1
                result[0] += right-r+1
            tmp = []
            l, r = left, mid+1
            while l < mid+1 or r < right+1:
                if r >= right+1 or (l < mid+1 and nums[l] <= nums[r]):
                    tmp.append(nums[l])
                    l += 1
                else:
                    tmp.append(nums[r])
                    r += 1
            nums[left:right+1] = tmp

        nums = [x-y for x, y in itertools.izip(nums1, nums2)]
        result = [0]
        merge_sort(nums, 0, len(nums)-1, result)
        return result[0]

# 2430 Hard 2430 Maximum Deletions on a String

In [ ]:
# Time:  O(n^2)
# Space: O(n)

# dp
class Solution(object):
    def deleteString(self, s):
        """
        :type s: str
        :rtype: int
        """
        if all(x == s[0] for x in s):
            return len(s)
        dp2 = [[0]*(len(s)+1) for i in xrange(2)]  # dp2[i%2][j]: max prefix length of s[i:] and s[j:]
        dp = [1]*len(s)  # dp[i]: max operation count of s[i:]
        for i in reversed(xrange(len(s)-1)):
            for j in xrange(i+1, len(s)):
                dp2[i%2][j] = dp2[(i+1)%2][j+1]+1 if s[j] == s[i] else 0
                if dp2[i%2][j] >= j-i:
                    dp[i] = max(dp[i], dp[j]+1)
        return dp[0]


# Time:  O(n^2)
# Space: O(n)
# dp, kmp algorithm
class Solution2(object):
    def deleteString(self, s):
        """
        :type s: str
        :rtype: int
        """
        def getPrefix(pattern, start):
            prefix = [-1]*(len(pattern)-start)
            j = -1
            for i in xrange(1, len(pattern)-start):
                while j != -1 and pattern[start+j+1] != pattern[start+i]:
                    j = prefix[j]
                if pattern[start+j+1] == pattern[start+i]:
                    j += 1
                prefix[i] = j
            return prefix

        if all(x == s[0] for x in s):
            return len(s)
        dp = [1]*len(s)  # dp[i]: max operation count of s[i:]
        for i in reversed(xrange(len(s)-1)):
            prefix = getPrefix(s, i)  # prefix[j]+1: longest prefix suffix length of s[i:j+1]
            for j in xrange(1, len(prefix), 2):
                if 2*(prefix[j]+1) == j+1:
                    dp[i] = max(dp[i], dp[i+(prefix[j]+1)]+1)
        return dp[0]


# Time:  O(n^2)
# Space: O(n)
# dp, rolling hash
class Solution3(object):
    def deleteString(self, s):
        """
        :type s: str
        :rtype: int
        """
        MOD, P = 10**9+7, (113, 109)
        def hash(i, j):
            return [(prefix[idx][j+1]-prefix[idx][i]*power[idx][j-i+1])%MOD for idx in xrange(len(P))]

        if all(x == s[0] for x in s):
            return len(s)

        power = [[1] for _ in xrange(len(P))]
        prefix = [[0] for _ in xrange(len(P))]
        for x in s:
            for idx, p in enumerate(P):
                power[idx].append((power[idx][-1]*p)%MOD)
                prefix[idx].append((prefix[idx][-1]*p+(ord(x)-ord('a')))%MOD)
        dp = [1]*len(s)  # dp[i]: max operation count of s[i:]
        for i in reversed(xrange(len(s)-1)):
            for j in xrange(1, (len(s)-i)//2+1):
                if hash(i, i+j-1) == hash(i+j, i+2*j-1):
                    dp[i] = max(dp[i], dp[i+j]+1)
        return dp[0]

# 2435 Hard 2435 Paths in Matrix Whose Sum Is Divisible by K

In [ ]:
# Time:  O(m * n * k)
# Space: O(n * k)

# dp
class Solution(object):
    def numberOfPaths(self, grid, k):
        """
        :type grid: List[List[int]]
        :type k: int
        :rtype: int
        """
        MOD = 10**9+7
        dp = [[0 for _ in xrange(k)] for _ in xrange(len(grid[0]))]
        dp[0][0] = 1
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                dp[j] = [((dp[j-1][(l-grid[i][j])%k] if j-1 >= 0 else 0)+dp[j][(l-grid[i][j])%k])%MOD for l in xrange(k)]
        return dp[-1][0]

# 2440 Hard 2440 Create Components With Same Value

In [ ]:
# Time:  O(n * sqrt(n))
# Space: O(n)

# bfs, greedy
class Solution(object):
    def componentValue(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def bfs(target):
            total = nums[:]
            lookup = [len(adj[u]) for u in xrange(len(adj))]
            q = [u for u in xrange(len(adj)) if lookup[u] == 1]
            while q:
                new_q = []
                for u in q:
                    if total[u] > target:
                        return False
                    if total[u] == target:
                        total[u] = 0
                    for v in adj[u]:
                        total[v] += total[u]
                        lookup[v] -= 1
                        if lookup[v] == 1:
                            new_q.append(v)
                q = new_q
            return True

        result = 0
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        total = sum(nums)
        for cnt in reversed(xrange(2, len(nums)+1)):
            if total%cnt == 0 and bfs(total//cnt):
                return cnt-1
        return 0


# Time:  O(n * sqrt(n))
# Space: O(n)
# iterative dfs, greedy
class Solution2(object):
    def componentValue(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def iter_dfs(target):
            total = nums[:]
            stk = [(1, (0, -1))]
            while stk:
                step, (u, p) = stk.pop()
                if step == 1:
                    stk.append((2, (u, p)))
                    for v in adj[u]:
                        if v == p:
                            continue
                        stk.append((1, (v, u)))
                elif step == 2:
                    for v in adj[u]:
                        if v == p:
                            continue
                        total[u] += total[v]
                    if total[u] == target:
                        total[u] = 0
            return total[0]

        result = 0
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        total = sum(nums)
        for cnt in reversed(xrange(2, len(nums)+1)):
            if total%cnt == 0 and iter_dfs(total//cnt) == 0:
                return cnt-1
        return 0


# Time:  O(n * sqrt(n))
# Space: O(n)
# dfs, greedy
class Solution3(object):
    def componentValue(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        def dfs(u, p, target):
            total = nums[u]
            for v in adj[u]:
                if v == p:
                    continue
                total += dfs(v, u, target)
            return total if total != target else 0

        result = 0
        adj = [[] for _ in xrange(len(nums))]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        total = sum(nums)
        for cnt in reversed(xrange(2, len(nums)+1)):
            if total%cnt == 0 and dfs(0, -1, total//cnt) == 0:
                return cnt-1
        return 0

# 2444 Hard 2444 Count Subarrays With Fixed Bounds

In [ ]:
# Time:  O(n)
# Space: O(1)

# two pointers
class Solution(object):
    def countSubarrays(self, nums, minK, maxK):
        """
        :type nums: List[int]
        :type minK: int
        :type maxK: int
        :rtype: int
        """
        result = left = 0
        right = [-1]*2
        for i, x in enumerate(nums):
            if not (minK <= x <= maxK):
                left = i+1
                continue
            if x == minK:
                right[0] = i
            if x == maxK:
                right[1] = i
            result += max(min(right)-left+1, 0)
        return result

# 2448 Hard 2448 Minimum Cost to Make Array Equal

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import itertools


# math, binary search
class Solution(object):
    def minCost(self, nums, cost):
        """
        :type nums: List[int]
        :type cost: List[int]
        :rtype: int
        """
        def f(x):
            return sum(abs(y-x)*c for y, c in itertools.izip(nums, cost))

        def check(x, t):
            return sum(c for y, c in itertools.izip(nums, cost) if y <= x) >= t
    
        idxs = range(len(nums))
        idxs.sort(key=lambda x: nums[x])
        left, right = 0, len(idxs)-1
        total = sum(cost)
        median = (total+1)//2
        while left <= right:
            mid = left+(right-left)//2
            if check(nums[idxs[mid]], median):
                right = mid-1
            else:
                left = mid+1
        return f(nums[idxs[left]])


# Time:  O(nlogn)
# Space: O(n)
import itertools


# binary search
class Solution2(object):
    def minCost(self, nums, cost):
        """
        :type nums: List[int]
        :type cost: List[int]
        :rtype: int
        """
        def f(x):
            return sum(abs(y-x)*c for y, c in itertools.izip(nums, cost))
    
        def check(x):
            return x+1 == len(idxs) or f(nums[idxs[x]]) < f(nums[idxs[x+1]])

        idxs = range(len(nums))
        idxs.sort(key=lambda x: nums[x])
        left, right = 0, len(idxs)-1
        while left <= right:
            mid = left+(right-left)//2
            if check(mid):
                right = mid-1
            else:
                left = mid+1
        return f(nums[idxs[left]])


# Time:  O(nlogn)
# Space: O(n)
# prefix sum
class Solution3(object):
    def minCost(self, nums, cost):
        """
        :type nums: List[int]
        :type cost: List[int]
        :rtype: int
        """
        idxs = range(len(nums))
        idxs.sort(key=lambda x: nums[x])
        prefix = [0]*(len(cost)+1)
        left = 0
        for i in xrange(len(cost)):
            if i-1 >= 0:
                left += prefix[i]*(nums[idxs[i]]-nums[idxs[i-1]])
            prefix[i+1] = prefix[i]+cost[idxs[i]]
        result = float("inf")
        suffix = right = 0
        for i in reversed(xrange(len(cost))):
            if i+1 < len(idxs):
                right += suffix*(nums[idxs[i+1]]-nums[idxs[i]])
            result = min(result, left+right)
            if i-1 >= 0:
                left -= prefix[i]*(nums[idxs[i]]-nums[idxs[i-1]])
            suffix += cost[idxs[i]]
        return result

# 2449 Hard 2449 Minimum Number of Operations to Make Arrays Similar

In [ ]:
# Time:  O(nlogn)
# Space: O(1)

import itertools


# greedy, sort
class Solution(object):
    def makeSimilar(self, nums, target):
        """
        :type nums: List[int]
        :type target: List[int]
        :rtype: int
        """
        nums.sort(key=lambda x: (x%2, x))
        target.sort(key=lambda x: (x%2, x))
        return sum(abs(x-y)//2 for x, y in itertools.izip(nums, target))//2

# 2454 Hard 2454 Next Greater Element IV

In [ ]:
# Time:  O(n)
# Space: O(n)

# mono stack
class Solution(object):
    def secondGreaterElement(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        result, stk1, stk2 = [-1]*len(nums), [], []
        for i, x in enumerate(nums):
            while stk2 and nums[stk2[-1]] < x:
                result[stk2.pop()] = x
            tmp = []
            while stk1 and nums[stk1[-1]] < x:
                tmp.append(stk1.pop())
            stk1.append(i)
            for x in reversed(tmp):
                stk2.append(x)
        return result

# 2458 Hard 2458 Height of Binary Tree After Subtree Removal Queries

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


class TreeNode(object):
    def __init__(self, val=0, left=None, right=None):
        pass


# iterative dfs
class Solution(object):
    def treeQueries(self, root, queries):
        """
        :type root: Optional[TreeNode]
        :type queries: List[int]
        :rtype: List[int]
        """
        def iter_dfs(root):
            top = collections.defaultdict(lambda: [0]*2)
            depth, height = {}, {}
            stk = [(1, (root, 0))]
            while stk:
                step, (curr, d) = stk.pop()
                if step == 1:
                    if not curr:
                        continue
                    stk.append((2, (curr, d)))
                    stk.append((1, (curr.right, d+1)))
                    stk.append((1, (curr.left, d+1)))
                elif step == 2:
                    h = 1+max((height[curr.left.val] if curr.left else 0), 
                              (height[curr.right.val] if curr.right else 0))
                    if h > top[d][0]:
                        top[d][0], top[d][1] = h, top[d][0]
                    elif h > top[d][1]:
                        top[d][1] = h
                    depth[curr.val], height[curr.val] = d, h
            return top, depth, height

        top, depth, height = iter_dfs(root)
        return [(depth[q]-1)+(top[depth[q]][0] if height[q] != top[depth[q]][0] else top[depth[q]][1]) for q in queries]


# Time:  O(n)
# Space: O(n)
import collections


# dfs
class Solution2(object):
    def treeQueries(self, root, queries):
        """
        :type root: Optional[TreeNode]
        :type queries: List[int]
        :rtype: List[int]
        """
        def dfs(curr, d):
            if not curr:
                return 0
            h = 1+max(dfs(curr.left, d+1), dfs(curr.right, d+1))
            if h > top[d][0]:
                top[d][0], top[d][1] = h, top[d][0]
            elif h > top[d][1]:
                top[d][1] = h
            depth[curr.val], height[curr.val] = d, h
            return h
        
        top = collections.defaultdict(lambda: [0]*2)
        depth, height = {}, {}
        dfs(root, 0)
        return [(depth[q]-1)+(top[depth[q]][0] if height[q] != top[depth[q]][0] else top[depth[q]][1]) for q in queries]

# 2459 Hard 2459 Sort Array by Moving Items to Empty Space

In [ ]:
# Time:  O(n)
# Space: O(n)

# greedy, sort
class Solution(object):
    def sortArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def min_moves(d):
            def index(x):
                return d*(len(nums)-1) if x == 0 else x-d

            lookup = [False]*len(nums)
            result = len(nums)
            for i in xrange(len(nums)):
                if lookup[nums[i]]:
                    continue
                l = 0
                while not lookup[nums[i]]:
                    lookup[nums[i]] = True
                    l += 1
                    i = index(nums[i])
                result -= 1
                if l >= 2:
                    result += 2
            return result-2*int(nums[d*(len(nums)-1)] != 0)

        return min(min_moves(0), min_moves(1))


# Time:  O(n)
# Space: O(n)
# greedy, sort
class Solution2(object):
    def sortArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def min_moves(d):
            def index(x):
                return d*(len(nums)-1) if x == 0 else x-d

            a = nums[:]
            result = 0
            for i in xrange(len(a)):
                l, has_zero = 1, (a[i] == 0)
                while index(a[i]) != i:
                    j = index(a[i])
                    a[i], a[j] = a[j], a[i]
                    l += 1
                    has_zero |= (a[i] == 0)
                if l >= 2:
                    result += l-1 if has_zero else l+1
            return result

        return min(min_moves(0), min_moves(1))

# 2463 Hard 2463 Minimum Total Distance Traveled

In [ ]:
# Time:  O(mlogm + nlogn + m * n)
# Space: O(n)

import collections


# sort, dp, prefix sum, mono deque
class Solution(object):
    def minimumTotalDistance(self, robot, factory):
        """
        :type robot: List[int]
        :type factory: List[List[int]]
        :rtype: int
        """
        robot.sort(), factory.sort()
        dp = [float("inf")]*(len(robot)+1)  # dp[j] at i: min of factory[:i+1] and robot[:j]
        dp[0] = 0
        for i in xrange(len(factory)):
            prefix = 0
            dq = collections.deque([(dp[0]-prefix, 0)])  # pattern of min in the sliding window with size (limit+1)
            for j in xrange(1, len(robot)+1):
                prefix += abs(robot[j-1]-factory[i][0])
                if j-dq[0][1] == factory[i][1]+1:
                    dq.popleft()
                while dq and dq[-1][0] >= dp[j]-prefix:
                    dq.pop()
                dq.append((dp[j]-prefix, j))
                dp[j] = dq[0][0]+prefix
        return dp[-1]


# Time:  O(mlogm + nlogn + m * n * l), l is the max limit
# Space: O(n)
import collections


# sort, dp
class Solution2(object):
    def minimumTotalDistance(self, robot, factory):
        """
        :type robot: List[int]
        :type factory: List[List[int]]
        :rtype: int
        """
        robot.sort(), factory.sort()
        dp = [float("inf")]*(len(robot)+1)  # dp[j] at i: min of factory[:i+1] and robot[:j]
        dp[0] = 0
        for i in xrange(len(factory)):
            for j in reversed(xrange(1, len(robot)+1)):
                curr = 0
                for k in xrange(min(factory[i][1], j)+1):
                    dp[j] = min(dp[j], dp[j-k]+curr)
                    if (j-1)-k >= 0:
                        curr += abs(robot[(j-1)-k]-factory[i][0])
        return dp[-1]

# 2468 Hard 2468 Split Message Based on Limit

In [ ]:
# Time:  O(n + rlogr), r is the number of messages
# Space: O(1)

# brute force, linear search (binary search doesn't work)
class Solution(object):
    def splitMessage(self, message, limit):
        """
        :type message: str
        :type limit: int
        :rtype: List[str]
        """
        cnt, l, total, base = 1, 1, len(message)+1, 1
        while 3+l*2 < limit:
            if total+(3+l)*cnt <= limit*cnt:
                break
            cnt += 1
            if cnt == base*10:
                l += 1
                base *= 10
            total += l
        if 3+l*2 >= limit:
            return []
        result = []
        j = 0
        for i in xrange(cnt):
            l = limit-(3+len(str(i+1))+len(str(cnt)))
            result.append("%s<%s/%s>"%(message[j:j+l], i+1, cnt))
            j += l
        return result

# 2478 Hard 2478 Number of Beautiful Partitions

In [ ]:
# Time:  O(n * k)
# Space: O(n)

# dp
class Solution(object):
    def beautifulPartitions(self, s, k, minLength):
        """
        :type s: str
        :type k: int
        :type minLength: int
        :rtype: int
        """
        MOD = 10**9+7
        PRIMES = {'2', '3', '5', '7'}
        dp = [0]*len(s)  # dp[i] at j : number of j beautiful partitions in s[:i+1] 
        for i in xrange(minLength-1, len(s)):
            if s[0] in PRIMES and s[i] not in PRIMES:
                dp[i] = 1
        for j in xrange(2, k+1):
            new_dp = [0]*len(s)
            curr = int(j == 1)
            for i in xrange(j*minLength-1, len(s)):
                if s[i-minLength+1] in PRIMES:
                    curr = (curr+dp[i-minLength])%MOD
                if s[i] not in PRIMES:
                    new_dp[i] = curr
            dp = new_dp
        return dp[-1]

# 2484 Hard 2484 Count Palindromic Subsequences

In [ ]:
# Time:  O(10^(l/2) * n), l = 5
# Space: O(10^(l/2) * n)

# freq table, prefix sum
class Solution(object):
    def countPalindromes(self, s):
        """
        :type s: str
        :rtype: int
        """
        MOD = 10**9+7
        cnt = [0]*10
        left = [[[0]*10 for _ in xrange(10)] for _ in xrange(len(s)+1)]
        for k in xrange(len(s)):
            left[k+1] = [[left[k][i][j] for j in xrange(10)] for i in xrange(10)]
            for i in xrange(10):
                left[k+1][int(s[k])][i] += cnt[i]
            cnt[int(s[k])] += 1
        cnt = [0]*10
        right = [[0]*10 for _ in xrange(10)]
        result = 0
        for k in reversed(xrange(len(s))):
            for i in xrange(10):
                for j in xrange(10):
                    result = (result+left[k][i][j]*right[i][j])%MOD
            for i in xrange(10):
                right[int(s[k])][i] += cnt[i]
            cnt[int(s[k])] += 1
        return result
                    

# Time:  O(10^(l/2) * n * l), l = 5
# Space: O(l)
# dp
class Solution2(object):
    def countPalindromes(self, s):
        """
        :type s: str
        :rtype: int
        """
        MOD = 10**9+7
        result = 0
        for i in xrange(10):
            for j in xrange(10):
                pattern = "%s%s*%s%s" % (i, j, j, i)
                dp = [0]*(5+1)
                dp[0] = 1
                for k in xrange(len(s)):
                    for l in reversed(xrange(5)):
                        if pattern[l] == '*' or pattern[l] == s[k]:
                            dp[l+1] = (dp[l+1]+dp[l])%MOD
                result = (result+dp[5])%MOD
        return result

# 2488 Hard 2488 Count Subarrays With Median K

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


# freq table, prefix sum
class Solution(object):
    def countSubarrays(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        idx = nums.index(k)
        lookup = collections.Counter()
        curr = 0
        for i in reversed(xrange(idx+1)):
            curr += 0 if nums[i] == k else -1 if nums[i] < k else +1
            lookup[curr] += 1
        result = curr = 0
        for i in xrange(idx, len(nums)):
            curr += 0 if nums[i] == k else -1 if nums[i] < k else +1
            result += lookup[-curr]+lookup[-(curr-1)]
        return result

# 2493 Hard 2493 Divide Nodes Into the Maximum Number of Groups

In [ ]:
# Time:  O(n^2)
# Space: O(n)

# iterative dfs, bfs
class Solution(object):
    def magnificentSets(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: int
        """
        def iter_dfs(u):
            group = []
            stk = [u]
            lookup[u] = 0
            while stk:
                u = stk.pop()
                group.append(u)
                for v in adj[u]:
                    if lookup[v] != -1:
                        if lookup[v] == lookup[u]:  # odd-length cycle, not bipartite
                            return []
                        continue
                    lookup[v] = lookup[u]^1
                    stk.append(v)
            return group

        def bfs(u):
            result = 0
            lookup = [False]*n
            q = [u]
            lookup[u] = True
            while q:
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if lookup[v]:
                            continue
                        lookup[v] = True
                        new_q.append(v)
                q = new_q
                result += 1
            return result
    
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u-1].append(v-1)
            adj[v-1].append(u-1)
        result = 0
        lookup = [-1]*n
        for u in xrange(n):
            if lookup[u] != -1:
                continue
            group = iter_dfs(u)
            if not group:
                return -1
            result += max(bfs(u) for u in group)
        return result


# Time:  O(n^2)
# Space: O(n)
# bfs
class Solution2(object):
    def magnificentSets(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: int
        """
        def bfs(u):
            group = []
            q = {u}
            lookup[u] = True
            while q:
                new_q = set()
                for u in q:
                    group.append(u)
                    for v in adj[u]:
                        if lookup[v]:
                            continue
                        lookup[v] = True
                        new_q.add(v)
                q = new_q
            return group
    
        def bfs2(u):
            result = 0
            lookup = [False]*n
            q = {u}
            lookup[u] = True
            while q:
                new_q = set()
                for u in q:
                    for v in adj[u]:
                        if v in q:
                            return 0
                        if lookup[v]:
                            continue
                        lookup[v] = True
                        new_q.add(v)
                q = new_q
                result += 1
            return result
    
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u-1].append(v-1)
            adj[v-1].append(u-1)
        result = 0
        lookup = [0]*n
        for u in xrange(n):
            if lookup[u]:
                continue
            group = bfs(u)
            mx = 0
            for u in group:
                d = bfs2(u)
                if d == 0:
                    return -1
                mx = max(mx, d)
            result += mx
        return result

# 2499 Hard 2499 Minimum Total Cost to Make Arrays Unequal

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections
import itertools


# greedy
class Solution(object):
    def minimumTotalCost(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: int
        """
        cnt = collections.Counter()
        result = 0
        for i, (x, y) in enumerate(itertools.izip(nums1, nums2)):
            if x != y:
                continue
            cnt[x] += 1
            result += i
        if not cnt:
            return 0
        majority = max(cnt.iterkeys(), key=lambda x: cnt[x])
        remain = cnt[majority]-(sum(cnt.itervalues())-cnt[majority])
        if remain <= 0:
            return result
        for i, (x, y) in enumerate(itertools.izip(nums1, nums2)):
            if x == y or majority in (x, y):
                continue
            result += i
            remain -= 1
            if not remain:
                return result
        return -1

# 2503 Hard 2503 Maximum Number of Points From Grid Queries

In [ ]:
# Time:  O((m * n + q) * log(m * n))
# Space: O(m * n)

import heapq


# bfs, heap, prefix sum, binary search
class Solution(object):
    def maxPoints(self, grid, queries):
        """
        :type grid: List[List[int]]
        :type queries: List[int]
        :rtype: List[int]
        """
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        min_heap = [(grid[0][0], 0, 0)]
        lookup = [[False]*len(grid[0]) for _ in xrange(len(grid))]
        lookup[0][0] = True
        mx = 0
        cnt = collections.Counter()
        while min_heap:
            curr, i, j = heapq.heappop(min_heap)
            mx = max(mx, curr)
            cnt[mx] += 1
            for di, dj in directions:
                ni, nj = i+di, j+dj
                if not (0 <= ni < len(grid) and
                        0 <= nj < len(grid[0]) and
                        not lookup[ni][nj]):
                    continue
                lookup[ni][nj] = True
                heapq.heappush(min_heap, (grid[ni][nj], ni, nj))
        vals = sorted(cnt.iterkeys())
        prefix = [0]*(len(vals)+1)
        for i in xrange(len(vals)):
            prefix[i+1] += prefix[i]+cnt[vals[i]]
        return map(lambda x: prefix[bisect.bisect_left(vals, x)], queries)

# 2508 Hard 2508 Add Edges to Make Degrees of All Nodes Even

In [ ]:
# Time:  O(n)
# Space: O(n)

# graph
class Solution(object):
    def isPossible(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: bool
        """
        adj = [set() for _ in xrange(n)]
        for u, v in edges:
            adj[u-1].add(v-1)
            adj[v-1].add(u-1)
        odds = [u for u in xrange(n) if len(adj[u])%2]
        if len(odds) == 0:
            return True
        if len(odds) == 2:
            return any(odds[0] not in adj[u] and odds[1] not in adj[u] for u in range(n))
        if len(odds) == 4:
            return ((odds[0] not in adj[odds[1]] and odds[2] not in adj[odds[3]]) or
                    (odds[0] not in adj[odds[2]] and odds[1] not in adj[odds[3]]) or
                    (odds[0] not in adj[odds[3]] and odds[1] not in adj[odds[2]]))
        return False

# 2509 Hard 2509 Cycle Length Queries in a Tree

In [ ]:
# Time:  O(q * n)
# Space: O(1)

# tree, lca
class Solution(object):
    def cycleLengthQueries(self, n, queries):
        """
        :type n: int
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        result = []
        for x, y in queries:
            cnt = 1
            while x != y:
                if x > y:
                    x, y = y, x
                y //= 2
                cnt += 1
            result.append(cnt)
        return result

# 2514 Hard 2514 Count Anagrams

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


# combinatorics
class Solution(object):
    def countAnagrams(self, s):
        """
        :type s: str
        :rtype: int
        """
        MOD = 10**9+7
        fact, inv, inv_fact = [[1]*2 for _ in xrange(3)]
        def lazy_init(n):
            while len(inv) <= n:  # lazy initialization
                fact.append(fact[-1]*len(inv) % MOD)
                inv.append(inv[MOD%len(inv)]*(MOD-MOD//len(inv)) % MOD)  # https://cp-algorithms.com/algebra/module-inverse.html
                inv_fact.append(inv_fact[-1]*inv[-1] % MOD)

        def factorial(n):
            lazy_init(n)
            return fact[n]

        def inv_factorial(n):
            lazy_init(n)
            return inv_fact[n]

        def count(j, i):
            result = 1
            cnt = collections.Counter()
            for k in  xrange(j, i+1):
                cnt[s[k]] += 1
            result = factorial(sum(cnt.itervalues()))
            for c in cnt.itervalues():
                result = (result*inv_factorial(c))%MOD
            return result

        result = 1
        j = 0
        for i in xrange(len(s)):
            if i+1 != len(s) and s[i+1] != ' ':
                continue
            result = (result*count(j, i))%MOD
            j = i+2
        return result

# 2518 Hard 2518 Number of Great Partitions

In [ ]:
# Time:  O(n * k)
# Space: O(k)

# knapsack dp
class Solution(object):
    def countPartitions(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        MOD = 10**9+7
        if sum(nums) < 2*k:
            return 0
        dp = [0]*k
        dp[0] = 1
        for x in nums:
            for i in reversed(xrange(k-x)):
                dp[i+x] = (dp[i+x]+dp[i])%MOD
        return (pow(2, len(nums), MOD)-2*reduce(lambda total, x: (total+x)%MOD, dp, 0))%MOD

# 2524 Hard 2524 Maximum Frequency Score of a Subarray

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


# two pointers, sliding window freq table, hash table
class Solution(object):
    def maxFrequencyScore(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        MOD = 10**9+7
        lookup = {}
        def powmod(n, p):
            if (n, p) not in lookup:
                lookup[n, p] = (lookup[n, p-1]*n)%MOD if p >= 2 else n%MOD  # assumed powmod(n, p-1) was accessed before powmod(n, p)
            return lookup[n, p]

        result = curr = 0
        cnt = collections.Counter()
        for i in xrange(len(nums)):
            if i >= k:
                curr = (curr-powmod(nums[i-k], cnt[nums[i-k]]))%MOD
                cnt[nums[i-k]] -= 1
                if cnt[nums[i-k]]:
                    curr = (curr+powmod(nums[i-k], cnt[nums[i-k]]))%MOD
            if cnt[nums[i]]:
               curr = (curr-powmod(nums[i], cnt[nums[i]]))%MOD
            cnt[nums[i]] += 1
            curr = (curr+powmod(nums[i], cnt[nums[i]]))%MOD
            if i >= k-1:
                result = max(result, curr)
        return result


# Time:  O(nlogn)
# Space: O(n)
import collections


# two pointers, sliding window, freq table
class Solution2(object):
    def maxFrequencyScore(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        MOD = 10**9+7
        result = curr = 0
        cnt = collections.Counter()
        for i in xrange(len(nums)):
            if i >= k:
                curr = (curr-pow(nums[i-k], cnt[nums[i-k]], MOD))%MOD
                cnt[nums[i-k]] -= 1
                if cnt[nums[i-k]]:
                    curr = (curr+pow(nums[i-k], cnt[nums[i-k]], MOD))%MOD
            if cnt[nums[i]]:
               curr = (curr-pow(nums[i], cnt[nums[i]], MOD))%MOD
            cnt[nums[i]] += 1
            curr = (curr+pow(nums[i], cnt[nums[i]], MOD))%MOD
            if i >= k-1:
                result = max(result, curr)
        return result

# 2528 Hard 2528 Maximize the Minimum Powered City

In [ ]:
# Time:  O(nlogk)
# Space: O(n)

# binary search, sliding window, greedy
class Solution(object):
    def maxPower(self, stations, r, k):
        """
        :type stations: List[int]
        :type r: int
        :type k: int
        :rtype: int
        """
        def min_power():
            mn = float("inf")
            curr = sum(stations[i] for i in xrange(r))
            for i in xrange(len(stations)):
                if i+r < len(stations):
                    curr += stations[i+r]
                if i >= r+1:
                    curr -= stations[i-(r+1)]
                mn = min(mn, curr)
            return mn
    
        def check(target):
            arr = stations[:]
            curr = sum(arr[i] for i in xrange(r))
            cnt = k
            for i in xrange(len(arr)):
                if i+r < len(arr):
                    curr += arr[i+r]
                if i >= r+1:
                    curr -= arr[i-(r+1)]
                if curr >= target:
                    continue
                diff = target-curr
                if diff > cnt:
                    return False
                cnt -= diff
                curr += diff
                if i+r < len(arr):
                    arr[i+r] += diff
            return True

        mn = min_power() 
        left, right = mn, mn+k
        while left <= right:
            mid = left + (right-left)//2
            if not check(mid):
                right = mid-1
            else:
                left = mid+1
        return right
    

# 2532 Hard 2532 Time to Cross a Bridge

In [ ]:
# Time:  O(k + nlogk)
# Space: O(k)

import heapq


# heap, simulation
class Solution(object):
    def findCrossingTime(self, n, k, time):
        """
        :type n: int
        :type k: int
        :type time: List[List[int]]
        :rtype: int
        """
        left_bridge, right_ware, right_bridge, left_ware = [(-(time[i][0]+time[i][2]), -i) for i in xrange(k)], [], [], []
        heapq.heapify(left_bridge)
        curr = 0
        while n:
            while left_ware and left_ware[0][0] <= curr:
                _, i = heapq.heappop(left_ware)
                heapq.heappush(left_bridge, (-(time[i][0]+time[i][2]), -i))
            while right_ware and right_ware[0][0] <= curr:
                _, i = heapq.heappop(right_ware)
                heapq.heappush(right_bridge, (-(time[i][0]+time[i][2]), -i))
            if right_bridge:
                _, i = heapq.heappop(right_bridge)
                i = -i
                curr += time[i][2]
                heapq.heappush(left_ware, (curr+time[i][3], i))
                n -= 1
            elif left_bridge and n-len(right_ware):
                _, i = heapq.heappop(left_bridge)
                i = -i
                curr += time[i][0]
                heapq.heappush(right_ware, (curr+time[i][1], i))
            else:
                curr = min(left_ware[0][0] if left_ware else float("inf"),
                           right_ware[0][0] if right_ware else float("inf"))
        return curr

# 2534 Hard 2534 Time Taken to Cross the Door

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections
import itertools


# queue, simulation
class Solution(object):
    def timeTaken(self, arrival, state):
        """
        :type arrival: List[int]
        :type state: List[int]
        :rtype: List[int]
        """
        def go_until(t):
            while curr[0] <= t and any(q):
                if not q[direction[0]]:
                    direction[0] ^= 1
                result[q[direction[0]].popleft()] = curr[0]
                curr[0] += 1
    
        UNKNOWN, ENTERING, EXITING = range(-1, 1+1)
        result = [0]*len(arrival)
        curr, direction = [float("-inf")], [UNKNOWN]
        q = [collections.deque(), collections.deque()]
        for i, (a, s) in enumerate(itertools.izip(arrival, state)):
            go_until(a-1)
            q[s].append(i)
            if not (a <= curr[0]):
                curr, direction = [a], [EXITING]
        go_until(float("inf"))
        return result

# 2538 Hard 2538 Difference Between Maximum and Minimum Price Sum

In [ ]:
# Time:  O(n)
# Space: O(n)

# iterative dfs, tree dp
class Solution(object):
    def maxOutput(self, n, edges, price):
        """
        :type n: int
        :type edges: List[List[int]]
        :type price: List[int]
        :rtype: int
        """
        def iter_dfs():
            result = 0
            stk = [(1, (0, -1, [price[0], 0]))]
            while stk:
                step, args = stk.pop()
                if step == 1:
                    u, p, ret = args
                    stk.append((2, (u, p, ret, 0)))
                elif step == 2:
                    u, p, ret, i = args
                    if i == len(adj[u]):
                        continue
                    stk.append((2, (u, p, ret, i+1)))
                    v = adj[u][i]
                    if v == p:
                        continue
                    new_ret = [price[v], 0]  # [max_path_sum, max_path_sum_without_last_node]
                    stk.append((3, (u, new_ret, ret)))
                    stk.append((1, (v, u, new_ret)))
                elif step == 3:
                    u, new_ret, ret = args
                    result = max(result, ret[0]+new_ret[1], ret[1]+new_ret[0])
                    ret[0] = max(ret[0], new_ret[0]+price[u])
                    ret[1] = max(ret[1], new_ret[1]+price[u])
            return result
        
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        return iter_dfs()


# Time:  O(n)
# Space: O(n)
# dfs, tree dp
class Solution2(object):
    def maxOutput(self, n, edges, price):
        """
        :type n: int
        :type edges: List[List[int]]
        :type price: List[int]
        :rtype: int
        """
        def dfs(u, p):
            dp = [price[u], 0]  # [max_path_sum, max_path_sum_without_last_node]
            for v in adj[u]:
                if v == p:
                    continue
                new_dp = dfs(v, u)
                result[0] = max(result[0], dp[0]+new_dp[1], dp[1]+new_dp[0])
                dp[0] = max(dp[0], new_dp[0]+price[u])
                dp[1] = max(dp[1], new_dp[1]+price[u])
            return dp
        
        result = [0]
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        dfs(0, -1)
        return result[0]


# Time:  O(n)
# Space: O(n)
# iterative dfs, tree dp
class Solution3(object):
    def maxOutput(self, n, edges, price):
        """
        :type n: int
        :type edges: List[List[int]]
        :type price: List[int]
        :rtype: int
        """
        def iter_dfs():
            dp = [0]*n  # max_sum
            stk = [(1, 0, -1)]
            while stk:
                step, u, p = stk.pop()
                if step == 1:
                    stk.append((2, u, p))
                    for v in adj[u]:
                        if v == p:
                            continue
                        stk.append((1, v, u))
                elif step == 2:
                    dp[u] = price[u]
                    for v in adj[u]:
                        if v == p:
                            continue
                        dp[u] = max(dp[u], dp[v]+price[u])
            return dp
        
        def iter_dfs2():
            result = 0
            stk = [(0, -1, 0)]
            while stk:
                u, p, curr = stk.pop()
                result = max(result, curr, dp[u]-price[u])
                top2 = [[curr, p], [0, -1]]
                for v in adj[u]:
                    if v == p:
                        continue
                    curr = [dp[v], v]
                    for i in xrange(len(top2)):
                        if curr > top2[i]:
                            top2[i], curr = curr, top2[i]
                for v in adj[u]:
                    if v == p:
                        continue
                    stk.append((v, u, (top2[0][0] if top2[0][1] != v else top2[1][0])+price[u]))
            return result
    
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        dp = iter_dfs()
        return iter_dfs2()


# Time:  O(n)
# Space: O(n)
# dfs, tree dp
class Solution4(object):
    def maxOutput(self, n, edges, price):
        """
        :type n: int
        :type edges: List[List[int]]
        :type price: List[int]
        :rtype: int
        """
        def dfs(u, p):
            dp[u] = price[u]
            for v in adj[u]:
                if v == p:
                    continue
                dp[u] = max(dp[u], dfs(v, u)+price[u])
            return dp[u]
        
        def dfs2(u, p, curr):
            result[0] = max(result[0], curr, dp[u]-price[u])
            top2 = [[curr, p], [0, -1]]
            for v in adj[u]:
                if v == p:
                    continue
                curr = [dp[v], v]
                for i in xrange(len(top2)):
                    if curr > top2[i]:
                        top2[i], curr = curr, top2[i]
            for v in adj[u]:
                if v == p:
                    continue
                dfs2(v, u, (top2[0][0] if top2[0][1] != v else top2[1][0])+price[u])
    
        result = [0]
        dp = [0]*n  # max_sum
        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        dfs(0, -1)
        dfs2(0, -1, 0)
        return result[0]

# 2543 Hard 2543 Check if Point Is Reachable

In [ ]:
# Time:  O(log(min(a, b)))
# Space: O(1)

# number theory
class Solution(object):
    def isReachable(self, targetX, targetY):
        """
        :type targetX: int
        :type targetY: int
        :rtype: bool
        """
        def gcd(a, b):
            while b:
                a, b = b, a%b
            return a
    
        g = gcd(targetX, targetY)
        return g == (g&~(g-1))  # co-prime other than factor 2

# 2547 Hard 2547 Minimum Cost to Split an Array

In [ ]:
# Time:  O(n^2)
# Space: O(n)

import collections


# dp
class Solution(object):
    def minCost(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        dp = [float("inf")]*(len(nums)+1)
        dp[0] = 0
        for i in xrange(len(dp)-1):
            cnt = [0]*len(nums)
            d = 0
            for j in xrange(i+1, len(dp)):
                cnt[nums[j-1]] += 1
                if cnt[nums[j-1]] == 1:
                    d += 1
                elif cnt[nums[j-1]] == 2:
                    d -= 1
                dp[j] = min(dp[j], dp[i]+k+((j-i)-d))
        return dp[-1]

# 2551 Hard 2551 Put Marbles in Bags

In [ ]:
# Time:  O(n) on average
# Space: O(1)

import random


# greedy, quick select
class Solution(object):
    def putMarbles(self, weights, k):
        """
        :type weights: List[int]
        :type k: int
        :rtype: int
        """
        def nth_element(nums, n, left=0, compare=lambda a, b: a < b):
            def tri_partition(nums, left, right, target, compare):
                mid = left
                while mid <= right:
                    if nums[mid] == target:
                        mid += 1
                    elif compare(nums[mid], target):
                        nums[left], nums[mid] = nums[mid], nums[left]
                        left += 1
                        mid += 1
                    else:
                        nums[mid], nums[right] = nums[right], nums[mid]
                        right -= 1
                return left, right

            right = len(nums)-1
            while left <= right:
                pivot_idx = random.randint(left, right)
                pivot_left, pivot_right = tri_partition(nums, left, right, nums[pivot_idx], compare)
                if pivot_left <= n <= pivot_right:
                    return
                elif pivot_left > n:
                    right = pivot_left-1
                else:  # pivot_right < n.
                    left = pivot_right+1

        for i in xrange(len(weights)-1):
            weights[i] += weights[i+1]
        weights.pop()
        result = 0
        nth_element(weights, (k-1)-1, compare=lambda a, b: a > b)
        result += sum(weights[i] for i in xrange(k-1))
        nth_element(weights, (k-1)-1)
        result -= sum(weights[i] for i in xrange(k-1))
        return result

# 2552 Hard 2552 Count Increasing Quadruplets

In [ ]:
# Time:  O(n^2)
# Space: O(n)

# dp
class Solution(object):
    def countQuadruplets(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        dp = [0]*len(nums)  # dp[j] at l: # of tuple (i, j, k) s.t. i < j < k < l and nums[i] < nums[k] < nums[j]
        result = 0
        for l in xrange(len(nums)):
            cnt = 0
            for j in xrange(l):
                if nums[j] < nums[l]:
                    cnt += 1
                    result += dp[j]
                elif nums[j] > nums[l]:
                    dp[j] += cnt
        return result

    
# Time:  O(n^2)
# Space: O(n^2)
# prefix sum
class Solution2(object):
    def countQuadruplets(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        right = [[0]*(len(nums)+1) for _ in xrange(len(nums))]
        for j in xrange(len(nums)):
            for i in reversed(xrange(j+1, len(nums))):
                right[j][i] = right[j][i+1] + int(nums[i] > nums[j])
        result = 0
        for k in xrange(len(nums)):
            left = 0
            for j in xrange(k):
                if nums[k] < nums[j]:
                    result += left*right[j][k+1]
                left += int(nums[k] > nums[j])
        return result


# Time:  O(n^2)
# Space: O(n^2)
# prefix sum
class Solution3(object):
    def countQuadruplets(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        left = [[0]*(len(nums)+1) for _ in xrange(len(nums))]
        for j in xrange(len(nums)):
            for i in xrange(j):
                left[j][i+1] = left[j][i] + int(nums[i] < nums[j])
        right = [[0]*(len(nums)+1) for _ in xrange(len(nums))]
        for j in xrange(len(nums)):
            for i in reversed(xrange(j+1, len(nums))):
                right[j][i] = right[j][i+1] + int(nums[i] > nums[j])
        result = 0
        for k in xrange(len(nums)):
            for j in xrange(k):
                if nums[k] < nums[j]:
                    result += left[k][j]*right[j][k+1]
        return result

# 2561 Hard 2561 Rearranging Fruits

In [ ]:
# Time:  O(n) on average
# Space: O(n)

import random
import collections


# freq table, greedy, quick select
class Solution(object):
    def minCost(self, basket1, basket2):
        """
        :type basket1: List[int]
        :type basket2: List[int]
        :rtype: int
        """
        def nth_element(nums, n, left=0, compare=lambda a, b: a < b):
            def tri_partition(nums, left, right, target, compare):
                mid = left
                while mid <= right:
                    if nums[mid] == target:
                        mid += 1
                    elif compare(nums[mid], target):
                        nums[left], nums[mid] = nums[mid], nums[left]
                        left += 1
                        mid += 1
                    else:
                        nums[mid], nums[right] = nums[right], nums[mid]
                        right -= 1
                return left, right
            
            right = len(nums)-1
            while left <= right:
                pivot_idx = random.randint(left, right)
                pivot_left, pivot_right = tri_partition(nums, left, right, nums[pivot_idx], compare)
                if pivot_left <= n <= pivot_right:
                    return
                elif pivot_left > n:
                    right = pivot_left-1
                else:  # pivot_right < n.
                    left = pivot_right+1
    
        cnt = collections.Counter()
        for x in basket1:
            cnt[x] += 1
        for x in basket2:
            cnt[x] -= 1
        mn = min(cnt.iterkeys())
        swaps = []
        for k, v in cnt.iteritems():
            if v%2:
                return -1
            swaps.extend(k for _ in xrange(abs(v)//2))
        nth_element(swaps, len(swaps)//2)
        return sum(min(swaps[i], mn*2) for i in xrange(len(swaps)//2))

# 2565 Hard 2565 Subsequence With the Minimum Score

In [ ]:
# Time:  O(n)
# Space: O(n)

# two pointers, dp
class Solution(object):
    def minimumScore(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: int
        """
        right = [-1]*len(s)  # right[i]: min removed rightmost index in s[i:]
        j = len(t)-1
        for i in reversed(xrange(len(s))):
            if j >= 0 and t[j] == s[i]:
                j -= 1
            right[i] = j
        result = j+1
        left = 0  # left at i: max removed leftmost index in s[:i]
        for i in xrange(len(s)):
            result = max(min(result, right[i]-left+1), 0)
            if left < len(t) and t[left] == s[i]:
                left += 1
        result = min(result, len(t)-left)
        return result

# 2569 Hard 2569 Handling Sum Queries After Update

In [ ]:
# Time:  O(nlogn + m + qlogn)
# Space: O(n)

# segment tree
class Solution(object):
    def handleQuery(self, nums1, nums2, queries):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        class SegmentTree(object):  # 0-based index
            def __init__(self, N,
                        build_fn=lambda _: 0,
                        query_fn=lambda x, y: y if x is None else max(x, y),
                        update_fn=lambda x, y: y if x is None else x+y):
                self.base = N
                self.H = (N-1).bit_length()
                self.query_fn = query_fn
                self.update_fn = update_fn
                self.tree = [None]*(2*N)
                self.lazy = [None]*N
                for i in xrange(self.base, self.base+N):
                    self.tree[i] = build_fn(i-self.base)
                for i in reversed(xrange(1, self.base)):
                    self.tree[i] = query_fn(self.tree[2*i], self.tree[2*i+1])

            def __apply(self, x, val):
                self.tree[x] = self.update_fn(self.tree[x], val)
                if x < self.base:
                    self.lazy[x] = self.update_fn(self.lazy[x], val)

            def update(self, L, R, h):  # Time: O(logN), Space: O(N)
                def pull(x):
                    while x > 1:
                        x >>= 1
                        self.tree[x] = self.query_fn(self.tree[x<<1], self.tree[(x<<1)+1])
                        if self.lazy[x] is not None:
                            self.tree[x] = self.update_fn(self.tree[x], self.lazy[x])

                if L > R:
                    return
                L += self.base
                R += self.base
                L0, R0 = L, R
                while L <= R:
                    if L & 1:  # is right child
                        self.__apply(L, h)
                        L += 1
                    if R & 1 == 0:  # is left child
                        self.__apply(R, h)
                        R -= 1
                    L >>= 1
                    R >>= 1
                pull(L0)
                pull(R0)

            def query(self, L, R):  # Time: O(logN), Space: O(N)
                def push(x):
                    n = self.H
                    while n:
                        y = x >> n
                        if self.lazy[y] is not None:
                            self.__apply(y<<1, self.lazy[y])
                            self.__apply((y<<1)+1, self.lazy[y])
                            self.lazy[y] = None
                        n -= 1

                result = None
                if L > R:
                    return result

                L += self.base
                R += self.base
                push(L)
                push(R)
                while L <= R:
                    if L & 1:  # is right child
                        result = self.query_fn(result, self.tree[L])
                        L += 1
                    if R & 1 == 0:  # is left child
                        result = self.query_fn(result, self.tree[R])
                        R -= 1
                    L >>= 1
                    R >>= 1
                return result

        st = SegmentTree(len(nums1),
                         build_fn=lambda i: (nums1[i], nums1[i]^1),
                         query_fn=lambda x, y: y if x is None else (x[0]+y[0], x[1]+y[1]),
                         update_fn=lambda x, y: y if x is None else (x[1], x[0]) if y == (1, 0) else x)
        result = []
        total = sum(nums2)
        for t, a, b in queries:
            if t == 1:
                st.update(a, b, (1, 0))
            elif t == 2:
                total += st.query(0, len(nums1)-1)[0]*a
            elif t == 3:
                result.append(total)
        return result

# 2573 Hard 2573 Find the String with LCP

In [ ]:
# Time:  O(n^2)
# Space: O(1)

# constructive algorithms, greedy, dp
class Solution(object):
    def findTheString(self, lcp):
        """
        :type lcp: List[List[int]]
        :rtype: str
        """
        result = [-1]*len(lcp)
        curr = 0
        for i in xrange(len(lcp)):
            if result[i] != -1:
                continue
            if curr == 26:
                return ""
            for j in xrange(i, len(lcp[0])):
                if lcp[i][j]:
                    result[j] = curr
            curr += 1
        for i in reversed(xrange(len(lcp))):
            for j in reversed(xrange(len(lcp[0]))):
                if lcp[i][j] != ((lcp[i+1][j+1]+1 if i+1 < len(lcp) and j+1 < len(lcp[0]) else 1) if result[i] == result[j] else 0):
                    return ''
        return "".join(map(lambda x: chr(ord('a')+x), result))

# 2577 Hard 2577 Minimum Time to Visit a Cell In a Grid

In [ ]:
# Time:  O(m * n * log(m * n))
# Space: O(m * n)

import heapq


# dijkstra's algorithm
class Solution(object):
    def minimumTime(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        DIRECTIONS = ((1, 0), (0, 1), (-1, 0), (0, -1))
        def dijkstra(start, target):
            best = [[float("inf")]*len(grid[0]) for _ in xrange(len(grid))]
            best[start[0]][start[1]] = 0
            min_heap = [(0, start[0], start[1])]
            while min_heap:
                curr, i, j = heapq.heappop(min_heap)
                if best[i][j] < curr:
                    continue
                if (i, j) == target:
                    break
                for di, dj in DIRECTIONS:  
                    ni, nj = i+di, j+dj   
                    if not (0 <= ni < len(grid) and 0 <= nj < len(grid[0]) and best[ni][nj] > max(grid[ni][nj]+int(grid[ni][nj]%2 == best[i][j]%2), curr+1)):
                        continue
                    best[ni][nj] = max(grid[ni][nj]+int(grid[ni][nj]%2 == best[i][j]%2), curr+1)
                    heapq.heappush(min_heap, (best[ni][nj], ni, nj))
            return best[target[0]][target[1]]

        if min(grid[0][1], grid[1][0]) > 1:
            return -1
        return dijkstra((0, 0), (len(grid)-1, len(grid[0])-1))

# 2581 Hard 2581 Count Number of Possible Root Nodes

In [ ]:
# Time:  O(n) 
# Space: O(h)

import collections


# iterative dfs
class Solution(object):
    def rootCount(self, edges, guesses, k):
        """
        :type edges: List[List[int]]
        :type guesses: List[List[int]]
        :type k: int
        :rtype: int
        """
        def iter_dfs():
            result = 0
            stk = [(0, -1)]
            while stk:
                u, p = stk.pop()
                result += int((p, u) in lookup)
                for v in adj[u]:
                    if v == p:
                        continue
                    stk.append((v, u))
            return result
        
        def iter_dfs2(curr):
            result = 0
            stk = [(0, -1, curr)]
            while stk:
                u, p, curr = stk.pop()
                if (p, u) in lookup:
                    curr -= 1
                if (u, p) in lookup:
                    curr += 1
                result += int(curr >= k)
                for v in adj[u]:
                    if v == p:
                        continue
                    stk.append((v, u, curr))
            return result

        adj = collections.defaultdict(list)
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        lookup = {(u, v) for u, v in guesses}
        curr = iter_dfs()
        return iter_dfs2(curr)


# Time:  O(n) 
# Space: O(h)
import collections


# dfs
class Solution2(object):
    def rootCount(self, edges, guesses, k):
        """
        :type edges: List[List[int]]
        :type guesses: List[List[int]]
        :type k: int
        :rtype: int
        """
        def dfs(u, p):
            cnt = int((p, u) in lookup)
            for v in adj[u]:
                if v == p:
                    continue
                cnt += dfs(v, u)
            return cnt
        
        def dfs2(u, p, curr):
            if (p, u) in lookup:
                curr -= 1
            if (u, p) in lookup:
                curr += 1
            cnt = int(curr >= k)
            for v in adj[u]:
                if v == p:
                    continue
                cnt += dfs2(v, u, curr)
            return cnt

        adj = collections.defaultdict(list)
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        lookup = {(u, v) for u, v in guesses}
        curr = dfs(0, -1)
        return dfs2(0, -1, curr)


# Time:  O(n) ~ O(n^2), worst case in star tree 
# Space: O(n)
import collections


# memoization
class Solution3(object):
    def rootCount(self, edges, guesses, k):
        """
        :type edges: List[List[int]]
        :type guesses: List[List[int]]
        :type k: int
        :rtype: int
        """
        cnt = [0]
        def memoization(u, p):
            if (u, p) not in memo:
                memo[u, p] = int((p, u) in lookup)
                for v in adj[u]:
                    if v == p:
                        continue
                    cnt[0] += 1
                    memo[u, p] += memoization(v, u)
            return memo[u, p]

        adj = collections.defaultdict(list)
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        lookup = {(u, v) for u, v in guesses}
        memo = {}
        return sum(memoization(i, -1) >= k for i in adj.iterkeys())

# 2584 Hard 2584 Split the Array to Make Coprime Products

In [ ]:
# Time:  O(n * sqrt(r)), r = max(nums)
# Space: O(sqrt(r))

import collections


# number theory
class Solution(object):
    def findValidSplit(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def factorize(x):
            result = []
            d = 2
            while d*d <= x:
                e = 0
                while x%d == 0:
                    x //= d
                    e += 1
                if e:
                    result.append([d, e])
                d += 1 if d == 2 else 2
            if x > 1:
                result.append([x, 1])
            return result
        
        right = collections.Counter()
        for x in reversed(nums):
            for p, c in factorize(x):
                right[p] += c
        left = collections.Counter()
        cnt = 0
        for i in xrange(len(nums)-1):
            for p, c in factorize(nums[i]):
                if not left[p]:
                    cnt += 1
                left[p] += c
                right[p] -= c
                if not right[p]:
                    cnt -= 1
            if not cnt:
                return i
        return -1

# 2585 Hard 2585 Number of Ways to Earn Points

In [ ]:
# Time:  O(n * t * c)
# Space: O(t)

# knapsack dp
class Solution(object):
    def waysToReachTarget(self, target, types):
        """
        :type target: int
        :type types: List[List[int]]
        :rtype: int
        """
        MOD = 10**9+7
        dp = [0]*(target+1)
        dp[0] = 1
        for c, m in types:
            for i in reversed(xrange(1, target+1)):
                for j in xrange(1, min(i//m, c)+1):
                    dp[i] = (dp[i]+dp[i-j*m])%MOD
        return dp[-1]


# Time:  O(n * t * c)
# Space: O(t)
# knapsack dp
class Solution2(object):
    def waysToReachTarget(self, target, types):
        """
        :type target: int
        :type types: List[List[int]]
        :rtype: int
        """
        MOD = 10**9+7
        dp = [0]*(target+1)
        dp[0] = 1
        for c, m in types:
            new_dp = [0]*(target+1)
            for i in xrange(target+1):
                for j in xrange(min((target-i)//m, c)+1):
                    new_dp[i+j*m] = (new_dp[i+j*m]+dp[i])%MOD
            dp = new_dp
        return dp[-1]

# 2589 Hard 2589 Minimum Time to Complete All Tasks

In [ ]:
# Time:  O(nlogn + n * r), r = max(e for _, e in tasks)
# Space: O(r)

# sort, greedy
class Solution(object):
    def findMinimumTime(self, tasks):
        """
        :type tasks: List[List[int]]
        :rtype: int
        """
        tasks.sort(key=lambda x: x[1])
        lookup = set()
        for s, e, d in tasks:
            d -= sum(i in lookup for i in xrange(s, e+1))
            for i in reversed(xrange(1, e+1)):
                if d <= 0:
                    break
                if i in lookup:
                    continue
                lookup.add(i)
                d -= 1
        return len(lookup)

# 2603 Hard 2603 Collect Coins in a Tree

In [ ]:
# Time:  O(n)
# Space: O(n)

# tree, bfs
class Solution(object):
    def collectTheCoins(self, coins, edges):
        """
        :type coins: List[int]
        :type edges: List[List[int]]
        :rtype: int
        """
        DISTANCE = 2

        adj = [set() for _ in xrange(len(coins))]
        for u, v in edges:
            adj[u].add(v)
            adj[v].add(u)
        n = len(coins)
        q = []
        for u in xrange(len(coins)):
            while len(adj[u]) == 1 and not coins[u]:
                v = adj[u].pop()
                adj[v].remove(u)
                n -= 1
                u = v
        q = [u for u in xrange(len(coins)) if len(adj[u]) == 1]
        for _ in xrange(DISTANCE):
            new_q = []
            for u in q:
                if not adj[u]:
                    assert(n == 1)
                    break
                v = adj[u].pop()
                adj[v].remove(u)
                n -= 1
                if len(adj[v]) == 1:
                    new_q.append(v)
            q = new_q
        return (n-1)*2

# 0010  Hard  10. Regular Expression Matching checkpoint

In [ ]:
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        # bottom up o(m*n)
        # https://leetcode.com/discuss/93024/easy-dp-java-solution-with-detailed-explanation
        if s == p:
            return True
        m, n = len(s), len(p)
        dp = [[False] * (n + 1) for _ in range(m + 1)]
        dp[0][0] = True
        for j in range(1, n):
            if p[j] == '*' and dp[0][j - 1]:
                dp[0][j + 1] = True
        # print dp
        for i in range(m):
            for j in range(n):
                if p[j] == '.' or p[j] == s[i]:
                    dp[i + 1][j + 1] = dp[i][j]
                elif p[j] == '*':
                    if p[j - 1] != s[i] and p[j - 1] != '.':
                        dp[i + 1][j + 1] = dp[i + 1][j - 1]
                    else:
                        dp[i + 1][j + 1] = dp[i + 1][j] or dp[i][j + 1] or dp[i + 1][j - 1]
        return dp[m][n]


if __name__ == '__main__':
    # begin
    s = Solution()
    print s.isMatch("", ".*")